In [ ]:
import uproot
import glob
import numpy as np
import pandas as pd
import time
import os
import re

# import pyvista as pv
# pv.set_jupyter_backend('trame')  # or 'panel' if using panel

from scipy.constants import epsilon_0, e as q_e
from scipy.interpolate import griddata
from scipy.optimize import curve_fit
from scipy.spatial import cKDTree

from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
from matplotlib.colorbar import ColorbarBase
from matplotlib.colors import Normalize
import matplotlib.colors as mcolors
from matplotlib.collections import LineCollection
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib as mpl

from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor, as_completed

import trimesh
import h5py
from trimesh.points import PointCloud

from common_functions import *
 
# Enable LaTeX rendering
mpl.rcParams['text.usetex'] = False
# Set the global font size
mpl.rcParams.update({'font.size': 12})

In [ ]:
## READ IN STACKED SPHERES GEOMETRY ## 

stacked_spheres = trimesh.load_mesh('../sphere-charging/geometry/stacked_spheres_frompython_cropped.stl') 
#stacked_spheres = trimesh.load_mesh('../sphere-charging/geometry/isolated_grains_interpolated.stl') 

# Visualize with Trimesh
stacked_spheres.show()

## Visual Representation of the Electric Field

### read raw fieldmap from build folder:

In [ ]:
iteration = 30

configIN = "onlyphotoemission"
directory = "/storage/scratch1/5/avira7/Grain-Charging-Simulation-Data/stacked-sphere/output111025/processed-fieldmaps"
#directory = "../build-dissipationRefinedGrid-initial8max0.8final12"

if iteration <10 :
    filenames = sorted(glob.glob(f"{directory}/fieldmaps/*00{iteration}*{configIN}*.txt")) #{iteration}
else:
    filenames = sorted(glob.glob(f"{directory}/fieldmaps/*{iteration}*{configIN}*.txt")) #{iteration}
print(filenames)

df  = read_data_format_efficient(filenames,scaling=True)

# check to make sure this matches the total nodes in outputlogs
#len(df[iteration]["E_mag"]) 

In [ ]:
## SETTINGS HERE ARE OPTIMIZED FOR ITERATION 86 ##

fieldIN = df[iteration]

N_DOWNSAMPLE_EMAG = 1
ARROW_VOXEL_SPACING = 0.02 
Y_SLICE = 0.0
THICKNESS = 0.001
VECTOR_SCALE_FACTOR = 9e-7 #2e-7 #2e-3 #5e-6 #2e-3 #5e-6 # Global scaling for glyphs
FIELD_AVERAGE_RADIUS = 2.8e-3 #2e-3

vmin, vmax = (-2e5, 2e5) # in log(E_mag) units
red_point = np.array([-0.1, 0, 0.1 - 0.015 + 0.037]) # 

# ----------------------------------------------------
# Voxel Downsampling Helper Function
# Ensures uniform spatial distribution of points in the slice
# ----------------------------------------------------
def voxel_downsample_points(points, spacing):
    """
    Selects one point per voxel defined by the spacing.
    Assumes points are 3D, but only uses X and Z for 2D density control.
    """
    # 1. Normalize coordinates to voxel indices (focus on X and Z for the 2D slice)
    min_x, _, min_z = points.min(axis=0)
    
    # Calculate bin indices for the points
    # We use X (column 0) and Z (column 2)
    x_indices = np.floor((points[:, 0] - min_x) / spacing).astype(int)
    z_indices = np.floor((points[:, 2] - min_z) / spacing).astype(int)
    
    # Combine X and Z indices into a unique hash/key
    max_x_index = x_indices.max() + 1
    voxel_keys = z_indices * max_x_index + x_indices

    # 2. Find the unique keys and their first occurrence
    # `return_index=True` gives the index of the first occurrence of each unique key
    unique_keys, unique_indices = np.unique(voxel_keys, return_index=True)
    
    return unique_indices

# ----------------------------------------------------
# Step 0: Load, Filter, and Downsample Data (Single Pass)
# ----------------------------------------------------
start_time = time.time()
points = fieldIN["pos"]
vectors = fieldIN["E"]
magnitudes = fieldIN["E_mag"]

# Apply initial filtering (z > 0 and magnitude > 0)
initial_mask = (points[:, 2] > 0) & (magnitudes > 0)
points = points[initial_mask]
vectors = vectors[initial_mask]
magnitudes = magnitudes[initial_mask]

# Aggressive Downsample (for point cloud, typically N_DOWNSAMPLE_EMAG=1 is best)
points_ds = points[::N_DOWNSAMPLE_EMAG]
vectors_ds = vectors[::N_DOWNSAMPLE_EMAG]
magnitudes_ds = magnitudes[::N_DOWNSAMPLE_EMAG]

# Create a PyVista Point Cloud (PolyData)
point_cloud = pv.PolyData(points_ds)
point_cloud["E_mag"] = magnitudes_ds   # Store log magnitude for visualization
point_cloud["Ex_val"] = vectors_ds[:,0] # Store vectors
point_cloud["Ez_val"] = vectors_ds[:,2] # Store vectors

print(f"Starting points (filtered by z > 0 & mag > 0): {len(points)}")

# ----------------------------------------------------
# Step 1: Geometry Setup and Slicing
# ----------------------------------------------------
start_time_geo = time.time()

# 1a. Load and Crop Geometry
pv_spheres = pv.PolyData(
    stacked_spheres.vertices,
    np.hstack([np.full((len(stacked_spheres.faces), 1), 3), stacked_spheres.faces])
).compute_normals()

# Define bounding box based on the downsampled field data
bbox_bounds = point_cloud.bounds
bbox = pv.Box(bounds=bbox_bounds)
pv_spheres_cropped = pv_spheres.clip_box(bbox, invert=False)

# 1b. Define the slice plane (ZX plane, normal along Y)
center = (point_cloud.center[0], Y_SLICE, point_cloud.center[2])
normal = [0, 1, 0] # ZX plane (normal along Y)

# Create a plane mesh for interpolation (this will be the magnitude slice)
plane_bounds = [
    point_cloud.bounds[0], point_cloud.bounds[1], # X bounds
    Y_SLICE, Y_SLICE,                             # Y (fixed)
    point_cloud.bounds[4], point_cloud.bounds[5]  # Z bounds
]

field_slice_mesh = pv.Plane(
    center=center, 
    direction=normal,
    j_size=bbox_bounds[1] - bbox_bounds[0], # X span
    i_size=bbox_bounds[5] - bbox_bounds[4], # Z span
    i_resolution=250, 
    j_resolution=250
)

# --- MODIFIED INTERPOLATION CALL FOR NEAREST NEIGHBOR ---
field_slice_interpolated = field_slice_mesh.interpolate(
    point_cloud,
    sharpness=3.0,      # High sharpness often helps with point data
    radius=0.001, #1e-12,       # Set radius to near-zero to minimize interpolation
    
    # 1. Provide a float placeholder to satisfy the TypeError
    null_value=1, 
    
    # 2. Force the strategy to use the nearest point (Nearest Neighbor)
    strategy='closest_point' # <--- This achieves the extrapolation you want
)
# --------------------------------------------------------


# Slice the geometry using the plane (more precise than slice_orthogonal)
geo_slice = pv_spheres_cropped.slice(normal=normal, origin=field_slice_mesh.center)
print(f"Geometry and slicing preparation complete in {time.time() - start_time_geo:.2f}s")

# ----------------------------------------------------
# Step 2: Vector Field Glyphs (Arrows)
# ----------------------------------------------------
start_time_vectors = time.time()

# 2a. Filter the downsampled points again to extract only those in the slice volume
# We use NumPy masking directly on the downsampled data (points_ds)
vector_mask = np.abs(points_ds[:, 1] - Y_SLICE) < THICKNESS
points_slice_full = points_ds[vector_mask]
vectors_slice_full = vectors_ds[vector_mask]
magnitudes_slice_full = magnitudes_ds[vector_mask]

# 2b. Apply Voxel Downsampling to achieve uniform density
unique_indices = voxel_downsample_points(points_slice_full, ARROW_VOXEL_SPACING)

points_slice = points_slice_full[unique_indices]
vectors_slice = vectors_slice_full[unique_indices]
magnitudes_slice = magnitudes_slice_full[unique_indices]

# ----------------------------------------------------
# MODIFICATION: Calculate Clamping Limit and Apply Clamping
# ----------------------------------------------------
# The maximum allowed length of an arrow is ARROW_VOXEL_SPACING.
# The glyph length = magnitude * VECTOR_SCALE_FACTOR * arrow_length_in_geom (which is 1.0 for pv.Arrow).
# To ensure: glyph_length <= ARROW_VOXEL_SPACING
# We need: magnitude * VECTOR_SCALE_FACTOR <= ARROW_VOXEL_SPACING
# Therefore: magnitude_clamped <= ARROW_VOXEL_SPACING / VECTOR_SCALE_FACTOR

# Define the maximum magnitude allowed
MAGNITUDE_MAX_CLAMP = ARROW_VOXEL_SPACING / VECTOR_SCALE_FACTOR /2

# Apply the clamping (upper bound) to the magnitude array
magnitudes_slice_clamped = np.clip(magnitudes_slice, a_min=None, a_max=MAGNITUDE_MAX_CLAMP)
# ----------------------------------------------------


# 2c. Create a PolyData object for glyphs
points_slice[:,1] = Y_SLICE - 2*THICKNESS# Force y-coordinate to the slice plane for visualization
vectors_slice[:,1] = 0.0 - 2* THICKNESS# Zero out Y component for 2D slice visualization
slice_mesh_vectors = pv.PolyData(points_slice)
slice_mesh_vectors['vectors'] = vectors_slice
# Use the CLAMPED magnitude array for scaling
slice_mesh_vectors['magnitude'] = magnitudes_slice_clamped
#slice_mesh_vectors['magnitude'] = np.log10(magnitudes_slice)

# # 2c. Create a PolyData object for glyphs
# points_slice[:,1] = Y_SLICE - 2*THICKNESS# Force y-coordinate to the slice plane for visualization
# vectors_slice[:,1] = 0.0 - 2* THICKNESS# Zero out Y component for 2D slice visualization
# slice_mesh_vectors = pv.PolyData(points_slice)
# slice_mesh_vectors['vectors'] = vectors_slice
# #slice_mesh_vectors['magnitude'] = np.log10(magnitudes_slice)
# slice_mesh_vectors['magnitude'] = magnitudes_slice

print(f"Points in vector slice (after density control): {len(points_slice)}, old length: {len(points_slice_full)}...")

# 2d. Create the glyphs
arrow = pv.Arrow(tip_length=0.3, tip_radius=0.2, shaft_radius=0.04)
glyphs = slice_mesh_vectors.glyph(
    orient='vectors',
    scale='magnitude',
    factor=VECTOR_SCALE_FACTOR,
    geom=arrow
)

# ----------------------------------------------------
# Step 3: Visualization
# ----------------------------------------------------
pl = pv.Plotter()
pl.set_background('white')

# Add interpolated magnitude slice
pl.add_mesh(
    field_slice_interpolated,
    scalars="Ex_val",
    cmap="YlGnBu",
    opacity=1,
    show_edges=False,
    clim=[vmin, vmax],   # <-- set fixed color range here
    # --- COLORBAR POSITIONING FIX ---
    scalar_bar_args={
        'title':None, # r'log$_{10}$(E$_{mag}$)', # Updated title format
        'vertical': False,            # Make it horizontal
        'position_x': 0.20,           # User-specified start position
        'position_y': 0.12,           # User-specified vertical position
        'width': 0.6,                 # User-specified width
        'height': 0.05,               # User-specified height
    }
    # -------------------------------
)

# Add sliced geometry (outline only)
pl.add_mesh(geo_slice, color="black", line_width=5,opacity=0.5)

# Add vector glyphs
pl.add_mesh(glyphs, color='black', show_scalar_bar=False, line_width=4,opacity=1)

# # Optional marker
sphere = pv.Sphere(radius=FIELD_AVERAGE_RADIUS, center=red_point)
pl.add_mesh(sphere, color="red", opacity=0.5)

# Force 2D (orthographic) projection and camera alignment for the ZX slice
pl.enable_parallel_projection()
pl.enable_2d_style()

# Align camera perpendicular to the slice
pl.view_xz() 

# --- ADD THIS LINE BEFORE pl.show() ---
pl.screenshot(f'figures/fieldvectors_iteration{iteration}.jpeg', scale=4)

# Show the plot
print(f"Total execution time: {time.time() - start_time:.2f}s")
pl.show(jupyter_backend='static') #jupyter_backend='static'

In [ ]:
fieldIN = df[iteration]

FIELD_AVERAGE_RADIUS = 2e-3
vmin, vmax = (-0.1, 0.1) # in log(E_mag) units
red_point = np.array([-0.1, 0, 0.1 - 0.015 + 0.037]) # 
red_point = np.array([0.1, 0., 0.1 - 0.015 + 0.037]) # 

# ----------------------------------------------------
# Step 0: Load, Filter, and Downsample Data (Single Pass)
# ----------------------------------------------------
start_time = time.time()
points = fieldIN["pos"]
vectors = fieldIN["E"]
magnitudes = fieldIN["E_mag"]

# Apply initial filtering (z > 0 and magnitude > 0)
initial_mask = (points[:, 2] > 0) & (magnitudes > 0)
points = points[initial_mask]
vectors = vectors[initial_mask]
magnitudes = magnitudes[initial_mask]

# --- Step: Interpolate Electric Field onto Surface and Compute Pressure ---
epsilon_0 = 8.854187817e-12  # Vacuum permittivity
 
# Interpolate full vector field onto surface geometry
field_cloud = pv.PolyData(points)
field_cloud["E_x"] = vectors[:, 0]
field_cloud["E_y"] = vectors[:, 1]
field_cloud["E_z"] = vectors[:, 2]

print(f"Starting points (filtered by z > 0 & mag > 0): {len(points)}")

# ----------------------------------------------------
# Step 1: Geometry Setup and Slicing 
# ----------------------------------------------------
start_time_geo = time.time()

# 1a. Load and Crop Geometry 
pv_spheres = pv.PolyData(
    stacked_spheres.vertices,
    np.hstack([np.full((len(stacked_spheres.faces), 1), 3), stacked_spheres.faces])
).compute_normals()
bbox_bounds = field_cloud.bounds
bbox = pv.Box(bounds=bbox_bounds)
pv_spheres_cropped = (
    pv_spheres
    .clip_box(bbox, invert=False)
    .extract_surface()
    .compute_normals(point_normals=True, cell_normals=True, inplace=False)
)

# Interpolate onto surface mesh
pv_spheres_interp = pv_spheres_cropped.interpolate(
    field_cloud,
    radius=0.002,
    strategy='closest_point',
    sharpness=3.0,
    null_value=0.0
)
 
# Extract interpolated field and surface normals
E_vec = np.stack([pv_spheres_interp["E_x"], pv_spheres_interp["E_y"], pv_spheres_interp["E_z"]], axis=1)
#E_vec = np.stack([pv_spheres_interp["E_x"], np.array([0]*len(pv_spheres_interp["E_x"])),np.array([0]*len(pv_spheres_interp["E_x"]))], axis=1)
 
# Compute dot product and magnitude squared
E_dot_n = np.einsum('ij,ij->i', E_vec, pv_spheres_interp.point_normals)
E_mag_sq = np.einsum('ij,ij->i', E_vec, E_vec)
 
# Maxwell stress tensor projection: normal pressure
electric_pressure = epsilon_0 * (E_dot_n**2 - 0.5 * E_mag_sq)
#electric_pressure = epsilon_0 * E_mag_sq # (E_dot_n**2) # - 0.5 * E_mag_sq)
pv_spheres_interp["electric_pressure"] = electric_pressure

print(f"Geometry and slicing preparation complete in {time.time() - start_time_geo:.2f}s")


pl = pv.Plotter()
pl.set_background('white')

# Add interpolated magnitude slice
pl.add_mesh(
    pv_spheres_interp,
    scalars="electric_pressure",
    cmap="seismic",
    opacity=1,
    show_edges=False,
    clim=[vmin, vmax],   # Use the defined log range
    interpolate_before_map=False,
    scalar_bar_args={
        # Title updated to reflect the log-magnitude plot, matching the data
        'title': None, #r'log$_{10}$(E$_{mag}$)', 
        'vertical': False,            
        'position_x': 0.22,           
        'position_y': 0.16,           
        'width': 0.6,                 
        'height': 0.05,               
    }
)

# Define camera: (position, focal_point, view_up)
pl.camera_position = [(-0.12166342033912077, -0.3800808894298424, 0.29897113376370393),
 (0.0, 0.0, 0.09238091282895766),
 (-0.08325442944763095, 0.4963396747392208, 0.8641270897600156)]
pl.camera_position =[(0.09973232154642885, -0.3416433894224343, 0.3806928562326606),
  (0.0, 0.0, 0.09238091282895766),
  (-0.13524196040858827, 0.6156566616046678, 0.776322411866769)]

# # Optional marker
sphere = pv.Sphere(radius=FIELD_AVERAGE_RADIUS, center=red_point)
pl.add_mesh(sphere, color="black", opacity=0.5)

#pl.view_isometric()  # nice angled 3D view
# or:
#pl.view_xy()  # top-down (Z up)
#pl.view_yz()  # side view
#pl.view_xz()  # front view

pl.show(jupyter_backend='static') #jupyter_backend='static'

In [ ]:


pl = pv.Plotter()
pl.set_background('white')

# Convert

# Add interpolated magnitude slice
pl.add_mesh(
    pv_spheres_interp,
    scalars="electric_pressure",
    cmap="seismic",
    opacity=1,
    clim=[-0.1, 0.1],   # Use the defined log range
    interpolate_before_map=False,
    show_edges=False,
    scalar_bar_args={
        # Title updated to reflect the log-magnitude plot, matching the data
        'title': None, #r'log$_{10}$(E$_{mag}$)', 
        'vertical': False,            
        'position_x': 0.22,           
        'position_y': 0.16,           
        'width': 0.6,                 
        'height': 0.05,               
    }
)

# Define camera: (position, focal_point, view_up)

# pl.camera_position =[(0.09973232154642885, -0.3416433894224343, 0.3806928562326606),
#  (0.0, 0.0, 0.09238091282895766),
#  (-0.13524196040858827, 0.6156566616046678, 0.776322411866769)]

pl.camera_position = [(0.1363737593585825, -0.11683469032939907, 0.47247186847252504),
 (0.0, 0.0, 0.09238091282895766),
 (-0.14259264054382748, 0.9305696518359464, 0.3372053705750612)]

# pl.camera_position= [(-0.12799331102842687, -0.2754039931467276, 0.29686110267708105),
#  (0.0, 0.0, 0.09238091282895766),
#  (-0.021089755556589956, 0.6022928135020578, 0.797996609650905)]

# # Optional marker
# sphere = pv.Sphere(radius=FIELD_AVERAGE_RADIUS, center=red_point)
# pl.add_mesh(sphere, color="black", opacity=0.5)

#pl.view_isometric()  # nice angled 3D view
# or:
#pl.view_xy()  # top-down (Z up)
#pl.view_yz()  # side view
#pl.view_xz()  # front view

pl.show(jupyter_backend='static') #jupyter_backend='static'

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as ticker
import numpy as np


# Enable LaTeX rendering
mpl.rcParams['text.usetex'] = False
# Set the global font size
mpl.rcParams.update({'font.size': 12})

cmap = plt.cm.YlGnBu 
vmin, vmax = (-2e5, 2e5) # in log(E_mag) units
norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)

fig, ax = plt.subplots(figsize=(6, 0.1))

# --- 1. Create and Configure the ScalarFormatter ---
formatter = ticker.ScalarFormatter(useMathText=True)

formatter.set_useOffset(False) 
formatter.set_powerlimits((0, 0)) 

# --- 2. Create the Colorbar and apply the Formatter ---
cb = mpl.colorbar.ColorbarBase(
    ax, 
    cmap=cmap, 
    norm=norm, 
    orientation='horizontal'
)

# Apply the formatter to the colorbar's x-axis
cb.ax.xaxis.set_major_formatter(formatter)

# --- 3. Display the Plot ---
plt.show()

In [ ]:
## SETTINGS HERE ARE OPTIMIZED FOR ITERATION 1 ##

N_DOWNSAMPLE_EMAG = 1
N_DOWNSAMPLE_BASE = 500 # This is the initial downsample applied to the full field before slicing

vmin, vmax = (-2e5, 2e5) #(-2e4, 2e4) 
VECTOR_SCALE_FACTOR = 1e-7 #2e-6 #5e-6 # Global scaling for glyphs

Y_SLICE = 0.0
THICKNESS = 0.002 # Original, thin slice thickness
EXCLUSION_DISTANCE = 0.011

FIELD_AVERAGE_RADIUS = 2e-3
red_point = np.array([-0.1, 0, 0.1 - 0.015+ 0.039]) #  
LOG_MAG_NULL_VALUE = 1.0 

fieldIN = df[iteration]

# ----------------------------------------------------
# Step 0: Load, Filter, and Downsample Data (Single Pass)
# ----------------------------------------------------
start_time = time.time()
points = fieldIN["pos"]
vectors = fieldIN["E"]
magnitudes = fieldIN["E_mag"]

# Apply initial filtering (z > 0 and magnitude > 0)
initial_mask = (points[:, 2] > 0) & (magnitudes > 0)
points = points[initial_mask]
vectors = vectors[initial_mask]
magnitudes = magnitudes[initial_mask]

if N_DOWNSAMPLE_EMAG > 1:
    print(f"Applying downsample of {N_DOWNSAMPLE_EMAG} to E_mag data.")
    #  Downsample (Base Downsample)
    points = points[::N_DOWNSAMPLE_EMAG]
    vectors = vectors[::N_DOWNSAMPLE_EMAG]
    magnitudes = magnitudes[::N_DOWNSAMPLE_EMAG]

# Create a PyVista Point Cloud (PolyData)
point_cloud = pv.PolyData(points)
point_cloud["E_mag"] = np.log10(magnitudes) # Store log magnitude for visualization
point_cloud["E_vec"] = vectors[:,0]             # Store vectors (E_x component)

print(f"Starting points (filtered by z > 0 & mag > 0): {len(points)}")

# ----------------------------------------------------
# Step 1: Geometry Setup and Slicing 
# ----------------------------------------------------
start_time_geo = time.time()

# 1a. Load and Crop Geometry 
pv_spheres = pv.PolyData(
    stacked_spheres.vertices,
    np.hstack([np.full((len(stacked_spheres.faces), 1), 3), stacked_spheres.faces])
).compute_normals()
bbox_bounds = point_cloud.bounds
bbox = pv.Box(bounds=bbox_bounds)
pv_spheres_cropped = pv_spheres.clip_box(bbox, invert=False)

# 1b. Define the slice plane
center = (point_cloud.center[0], Y_SLICE, point_cloud.center[2])
normal = [0, 1, 0] # ZX plane (normal along Y)

plane_bounds = [
    point_cloud.bounds[0], point_cloud.bounds[1], # X bounds
    Y_SLICE, Y_SLICE,                             # Y (fixed)
    point_cloud.bounds[4], point_cloud.bounds[5]  # Z bounds
]

field_slice_mesh = pv.Plane(
    center=center, 
    direction=normal,
    j_size=bbox_bounds[1] - bbox_bounds[0], 
    i_size=bbox_bounds[5] - bbox_bounds[4], 
    i_resolution=250, 
    j_resolution=250
)

# --- MODIFIED INTERPOLATION CALL FOR NEAREST NEIGHBOR ---
field_slice_interpolated = field_slice_mesh.interpolate(
    point_cloud,
    sharpness=3.0,      # High sharpness often helps with point data
    radius=0.002, #1e-12,       # Set radius to near-zero to minimize interpolation
    
    # 1. Provide a float placeholder to satisfy the TypeError
    null_value=1, 
    
    # 2. Force the strategy to use the nearest point (Nearest Neighbor)
    strategy='closest_point' # <--- This achieves the extrapolation you want
)

geo_slice = pv_spheres_cropped.slice(normal=normal, origin=field_slice_mesh.center)
print(f"Geometry and slicing preparation complete in {time.time() - start_time_geo:.2f}s")

# ----------------------------------------------------
## Step 2: Filter Data Based on Distance to geo_slice
# ----------------------------------------------------
start_time_filter = time.time()

# --- 2a. Thin Slice Filtering ---
vector_mask = np.abs(points[:, 1] - Y_SLICE) < THICKNESS
points_slice = points[vector_mask]
vectors_slice = vectors[vector_mask]
magnitudes_slice = magnitudes[vector_mask]

# Aggressive Downsample (Base Downsample)
points_slice_ds = points_slice[::N_DOWNSAMPLE_BASE]
vectors_slice_ds = vectors_slice[::N_DOWNSAMPLE_BASE]
magnitudes_slice_ds = magnitudes_slice[::N_DOWNSAMPLE_BASE]

# 2. Find the distance to the nearest neighbor in geo_slice for every point in points_ds
# We use query with k=1 (nearest neighbor)
tree = cKDTree(geo_slice.points)
distance_to_geometry, _ = tree.query(points_slice_ds, k=1)

# 3. Create a mask: keep only points whose distance is greater than the threshold
keep_mask = distance_to_geometry > EXCLUSION_DISTANCE

# 4. Apply the mask to the base data arrays
points_slice_ds = points_slice_ds[keep_mask]
vectors_slice_ds = vectors_slice_ds[keep_mask]
magnitudes_slice_ds = magnitudes_slice_ds[keep_mask]          

# 2b. Create PolyData for thin glyphs
#points_slice_ds[:,1] = Y_SLICE  -THICKNESS*2# Force y-coordinate to the slice plane for visualization
slice_mesh_vectors_noRim = pv.PolyData(points_slice_ds)
slice_mesh_vectors_noRim['vectors'] = vectors_slice_ds
slice_mesh_vectors_noRim['magnitude'] = magnitudes_slice_ds

# 2c. Create the THIN glyphs
arrow = pv.Arrow(tip_length=0.2, tip_radius=0.08, shaft_radius=0.02)
glyphs = slice_mesh_vectors_noRim.glyph(
    orient='vectors',
    scale='magnitude',
    factor=VECTOR_SCALE_FACTOR,
    geom=arrow
)

print(f"Distance filtering removed {len(keep_mask) - len(points_slice_ds)} points. New length: {len(points_slice_ds)}")
print(f"Vector field preparation complete in {time.time() - start_time_filter:.2f}s")

# ----------------------------------------------------
## Step 3: Visualization 📊
# ----------------------------------------------------
pl = pv.Plotter()
pl.set_background('white')

# Add interpolated magnitude slice
pl.add_mesh(
    field_slice_interpolated,
    scalars="E_vec",
    cmap="viridis",
    opacity=0.9,
    show_edges=False,
    clim=[vmin, vmax],   # Use the defined log range
    scalar_bar_args={
        # Title updated to reflect the log-magnitude plot, matching the data
        'title': None, #r'log$_{10}$(E$_{mag}$)', 
        'vertical': False,            
        'position_x': 0.22,           
        'position_y': 0.16,           
        'width': 0.6,                 
        'height': 0.05,               
    }
)

# Add sliced geometry 
pl.add_mesh(geo_slice, color="black", line_width=2.5)

# Add vector glyphs
pl.add_mesh(glyphs, color='white', show_scalar_bar=False)
#pl.add_mesh(glyphs, show_scalar_bar=False, cmap='coolwarm') # color='white', show_scalar_bar=False)

# Optional marker 
#sphere = pv.Sphere(radius=FIELD_AVERAGE_RADIUS, center=red_point)
#pl.add_mesh(sphere, color="red", opacity=0.5)

# Force 2D (orthographic) projection and camera alignment for the ZX slice
pl.enable_parallel_projection()
pl.enable_2d_style()
pl.view_xz()

# Show the plot
print(f"Total execution time: {time.time() - start_time:.2f}s")
pl.show(jupyter_backend='static')

In [ ]:
## SETTINGS HERE ARE OPTIMIZED FOR ITERATION 86 ##

N_DOWNSAMPLE_EMAG = 1
N_DOWNSAMPLE_BASE = 500 # This is the initial downsample applied to the full field before slicing

vmin, vmax = (-2e5, 2e5) 
VECTOR_SCALE_FACTOR = 1e-7 # Global scaling for glyphs

Y_SLICE = 0.0
THICKNESS = 0.002 # Original, thin slice thickness
EXCLUSION_DISTANCE = 0.011

FIELD_AVERAGE_RADIUS = 2e-3
red_point = np.array([-0.1, 0, 0.1 - 0.015 + 0.037]) # 
LOG_MAG_NULL_VALUE = 1.0 

fieldIN = df[iteration]

# ----------------------------------------------------
# Step 0: Load, Filter, and Downsample Data (Single Pass)
# ----------------------------------------------------
start_time = time.time()
points = fieldIN["pos"]
vectors = fieldIN["E"]
magnitudes = fieldIN["E_mag"]

# Apply initial filtering (z > 0 and magnitude > 0)
initial_mask = (points[:, 2] > 0) & (magnitudes > 0)
points = points[initial_mask]
vectors = vectors[initial_mask]
magnitudes = magnitudes[initial_mask]

if N_DOWNSAMPLE_EMAG > 1:
    print(f"Applying downsample of {N_DOWNSAMPLE_EMAG} to E_mag data.")
    #  Downsample (Base Downsample)
    points = points[::N_DOWNSAMPLE_EMAG]
    vectors = vectors[::N_DOWNSAMPLE_EMAG]
    magnitudes = magnitudes[::N_DOWNSAMPLE_EMAG]

# Create a PyVista Point Cloud (PolyData)
point_cloud = pv.PolyData(points)
point_cloud["E_mag"] = np.log10(magnitudes) # Store log magnitude for visualization
point_cloud["E_vec"] = vectors[:,0]             # Store vectors (E_x component)

print(f"Starting points (filtered by z > 0 & mag > 0): {len(points)}")

# ----------------------------------------------------
# Step 1: Geometry Setup and Slicing 
# ----------------------------------------------------
start_time_geo = time.time()

# 1a. Load and Crop Geometry 
pv_spheres = pv.PolyData(
    stacked_spheres.vertices,
    np.hstack([np.full((len(stacked_spheres.faces), 1), 3), stacked_spheres.faces])
).compute_normals()
bbox_bounds = point_cloud.bounds
bbox = pv.Box(bounds=bbox_bounds)
pv_spheres_cropped = pv_spheres.clip_box(bbox, invert=False)

# 1b. Define the slice plane
center = (point_cloud.center[0], Y_SLICE, point_cloud.center[2])
normal = [0, 1, 0] # ZX plane (normal along Y)

plane_bounds = [
    point_cloud.bounds[0], point_cloud.bounds[1], # X bounds
    Y_SLICE, Y_SLICE,                             # Y (fixed)
    point_cloud.bounds[4], point_cloud.bounds[5]  # Z bounds
]

field_slice_mesh = pv.Plane(
    center=center, 
    direction=normal,
    j_size=bbox_bounds[1] - bbox_bounds[0], 
    i_size=bbox_bounds[5] - bbox_bounds[4], 
    i_resolution=250, 
    j_resolution=250
)

# --- MODIFIED INTERPOLATION CALL FOR NEAREST NEIGHBOR ---
field_slice_interpolated = field_slice_mesh.interpolate(
    point_cloud,
    sharpness=3.0,      # High sharpness often helps with point data
    radius=0.002, #1e-12,       # Set radius to near-zero to minimize interpolation
    
    # 1. Provide a float placeholder to satisfy the TypeError
    null_value=1, 
    
    # 2. Force the strategy to use the nearest point (Nearest Neighbor)
    strategy='closest_point' # <--- This achieves the extrapolation you want
)

geo_slice = pv_spheres_cropped.slice(normal=normal, origin=field_slice_mesh.center)
print(f"Geometry and slicing preparation complete in {time.time() - start_time_geo:.2f}s")

# ----------------------------------------------------
## Step 2: Filter Data Based on Distance to geo_slice
# ----------------------------------------------------
start_time_filter = time.time()

# --- 2a. Thin Slice Filtering ---
vector_mask = np.abs(points[:, 1] - Y_SLICE) < THICKNESS
points_slice = points[vector_mask]
vectors_slice = vectors[vector_mask]
magnitudes_slice = magnitudes[vector_mask]

# Aggressive Downsample (Base Downsample)
points_slice_ds = points_slice[::N_DOWNSAMPLE_BASE]
vectors_slice_ds = vectors_slice[::N_DOWNSAMPLE_BASE]
magnitudes_slice_ds = magnitudes_slice[::N_DOWNSAMPLE_BASE]

# 2. Find the distance to the nearest neighbor in geo_slice for every point in points_ds
# We use query with k=1 (nearest neighbor)
tree = cKDTree(geo_slice.points)
distance_to_geometry, _ = tree.query(points_slice_ds, k=1)

# 3. Create a mask: keep only points whose distance is greater than the threshold
keep_mask = distance_to_geometry > EXCLUSION_DISTANCE

# 4. Apply the mask to the base data arrays
points_slice_ds = points_slice_ds[keep_mask]
vectors_slice_ds = vectors_slice_ds[keep_mask]
magnitudes_slice_ds = magnitudes_slice_ds[keep_mask]          

# 2b. Create PolyData for thin glyphs
points_slice_ds[:,1] = Y_SLICE  # Force y-coordinate to the slice plane for visualization
slice_mesh_vectors_noRim = pv.PolyData(points_slice_ds)
slice_mesh_vectors_noRim['vectors'] = vectors_slice_ds
slice_mesh_vectors_noRim['magnitude'] = magnitudes_slice_ds

# 2c. Create the THIN glyphs
arrow = pv.Arrow(tip_length=0.2, tip_radius=0.08, shaft_radius=0.01)
glyphs = slice_mesh_vectors_noRim.glyph(
    orient='vectors',
    scale='magnitude',
    factor=VECTOR_SCALE_FACTOR,
    geom=arrow
)

print(f"Distance filtering removed {len(keep_mask) - len(points_slice_ds)} points. New length: {len(points_slice_ds)}")
print(f"Vector field preparation complete in {time.time() - start_time_filter:.2f}s")

# ----------------------------------------------------
## Step 3: Visualization 📊
# ----------------------------------------------------
pl = pv.Plotter()
pl.set_background('white')

# Add interpolated magnitude slice
pl.add_mesh(
    field_slice_interpolated,
    scalars="E_vec",
    cmap="viridis",
    opacity=0.9,
    show_edges=False,
    clim=[vmin, vmax],   # Use the defined log range
    scalar_bar_args={
        # Title updated to reflect the log-magnitude plot, matching the data
        'title': None, #r'log$_{10}$(E$_{mag}$)', 
        'vertical': False,            
        'position_x': 0.22,           
        'position_y': 0.16,           
        'width': 0.6,                 
        'height': 0.05,               
    }
)

# Add sliced geometry 
pl.add_mesh(geo_slice, color="black", line_width=2.5)

# Add vector glyphs
pl.add_mesh(glyphs, color='white', show_scalar_bar=False)
#pl.add_mesh(glyphs, show_scalar_bar=False, cmap='coolwarm') # color='white', show_scalar_bar=False)

# Optional marker 
sphere = pv.Sphere(radius=FIELD_AVERAGE_RADIUS, center=red_point)
pl.add_mesh(sphere, color="red", opacity=0.5)

# Force 2D (orthographic) projection and camera alignment for the ZX slice
pl.enable_parallel_projection()
pl.enable_2d_style()
pl.view_xz()

# Show the plot
print(f"Total execution time: {time.time() - start_time:.2f}s")
pl.show(jupyter_backend='static') #jupyter_backend='static'

In [ ]:
from typing import Tuple

def filter_coordinates(
    df: pd.DataFrame, 
    column_name: str, 
    x_range: Tuple[float, float], 
    y_range: Tuple[float, float], 
    z_range: Tuple[float, float]
) -> pd.Series:
    """
    Filters a DataFrame Series containing coordinate strings based on specified 
    (x, y, z) ranges.

    The coordinates in the column_name are expected to be formatted as strings 
    like "(x, y, z)".

    Args:
        df (pd.DataFrame): The input DataFrame.
        column_name (str): The name of the column containing the coordinate strings.
        x_range (Tuple[float, float]): (min, max) for the x-coordinate filter.
        y_range (Tuple[float, float]): (min, max) for the y-coordinate filter.
        z_range (Tuple[float, float]): (min, max) for the z-coordinate filter.

    Returns:
        pd.Series: A pandas Series containing only the coordinate strings that
                   fall within all three specified ranges.
    """
    
    coords = np.vstack( df[column_name])
    
    # X-Filter: x is greater than x_min AND x is less than x_max
    x_min, x_max = x_range
    x_filter = (coords[:,0] > x_min) & (coords[:,0]  < x_max)

    # Y-Filter: y is greater than y_min AND y is less than y_max
    y_min, y_max = y_range
    y_filter = (coords[:,1] > y_min) & (coords[:,1]  < y_max)

    # Z-Filter: z is greater than z_min AND z is less than z_max
    z_min, z_max = z_range
    z_filter = (coords[:,2] > z_min) & (coords[:,2]  < z_max)

    # --- 3. Combine the filters and apply to the original series ---
    combined_filter = x_filter & y_filter & z_filter

    # Return the original coordinate strings that match the combined filter
    return df[column_name][combined_filter]

In [ ]:
np.vstack(all_gamma_holes_df["Post_Step_Position_mm"])

In [ ]:
# --- Example Usage ---

# Define the filtering ranges based on your request
X_RANGE = (plane_bounds[0],plane_bounds[1])
Y_RANGE = (-THICKNESS/2, THICKNESS/2)
Z_RANGE = (plane_bounds[4],plane_bounds[5])

# Call the function with the sample data and required ranges
filtered_series_gamma = filter_coordinates(df=all_gamma_holes_df,column_name="Post_Step_Position_mm",
    x_range=X_RANGE,y_range=Y_RANGE,z_range=Z_RANGE)
print(f"\nTotal points found: {len(filtered_series_gamma)}, starting length: {len(all_gamma_holes_df)}")

# Call the function with the sample data and required ranges
filtered_series_electron = filter_coordinates(df=all_electrons_inside_df,column_name="Post_Step_Position_mm",
    x_range=X_RANGE,y_range=Y_RANGE,z_range=Z_RANGE)
print(f"\nTotal points found: {len(filtered_series_electron)}, starting length: {len(all_electrons_inside_df)}")

In [ ]:
# ----------------------------------------------------
## Step 3: Visualization 📊
# ----------------------------------------------------
pl = pv.Plotter()
pl.set_background('white')

# Add interpolated magnitude slice
pl.add_mesh(
    field_slice_interpolated,
    scalars="E_vec",
    cmap="viridis",
    opacity=0.9,
    show_edges=False,
    clim=[vmin, vmax],   # Use the defined log range
    scalar_bar_args={
        # Title updated to reflect the log-magnitude plot, matching the data
        'title': None, #r'log$_{10}$(E$_{mag}$)', 
        'vertical': False,            
        'position_x': 0.22,           
        'position_y': 0.16,           
        'width': 0.6,                 
        'height': 0.05,               
    }
)

# Add sliced geometry 
pl.add_mesh(geo_slice, color="black", line_width=2.5)

# Add vector glyphs
pl.add_mesh(glyphs, color='white', show_scalar_bar=False)
#pl.add_mesh(glyphs, show_scalar_bar=False, cmap='coolwarm') # color='white', show_scalar_bar=False)

# Optional marker 
sphere = pv.Sphere(radius=FIELD_AVERAGE_RADIUS, center=red_point)
pl.add_mesh(sphere, color="red", opacity=0.5)


# # Add the single PolyData mesh
# pl.add_mesh(
#     pv.PolyData(np.vstack(filtered_series_gamma)),
#     color='red',
#     opacity=0.9,
#     # Key arguments for fast point visualization:
#     render_points_as_spheres=True,  # Makes them look like spheres
#     point_size=3,                  # Controls the size of the 'spheres'
#     # The 'point_size' units are in screen pixels by default
# )

# Add the single PolyData mesh
pl.add_mesh(
    pv.PolyData(np.vstack(filtered_series_electron)),
    color='red',
    opacity=0.9,
    # Key arguments for fast point visualization:
    render_points_as_spheres=True,  # Makes them look like spheres
    point_size=3,                  # Controls the size of the 'spheres'
    # The 'point_size' units are in screen pixels by default
)

# Force 2D (orthographic) projection and camera alignment for the ZX slice
pl.enable_parallel_projection()
pl.enable_2d_style()
pl.view_xz()

# Show the plot
print(f"Total execution time: {time.time() - start_time:.2f}s")
pl.show(jupyter_backend='static') #jupyter_backend='static'

In [ ]:
configIN = "onlyphotoemission"
directory_path =  "../build-smallerworld-initial8max0.8final12/root/" #"../build-adaptive-barns-fixed/root/"
filelist = sorted(glob.glob(f"{directory_path}/*iteration*{configIN}*.root"))

all_gamma_holes = []
all_electrons_inside = []

for fileIN in filelist:
    print(fileIN.split("/")[-1])
    number_str = fileIN.split("/")[-1].split("_")[1]
    iterationNUM = int(''.join(filter(str.isdigit, number_str)))

    if iterationNUM > 5:
        break

    # read data from different iterations
    vars()["gamma_holes_"+str(number_str)], vars()["electrons_inside_"+str(number_str)], _ = calculate_stats(read_rootfile(fileIN.split("/")[-1], directory_path=directory_path),
                                                                                                             config=configIN)
    
    # surf, ilm_values = plot_face_illumination(vars()["gamma_holes_"+str(number_str)], stacked_spheres, vmin=0, vmax=1)
    # # Make sure each triangle has its own unique vertices
    # surface_edited = surface.copy()
    # surface_edited.unmerge_vertices()
    # surface_edited.visual.vertex_colors = None
    # surface_edited.show()

    all_gamma_holes.append(vars()["gamma_holes_"+str(number_str)])
    all_electrons_inside.append(vars()["electrons_inside_"+str(number_str)])

# Concatenate all iterations into single DataFrames
all_gamma_holes_df = pd.concat(all_gamma_holes, ignore_index=True)
all_electrons_inside_df = pd.concat(all_electrons_inside, ignore_index=True)

In [ ]:
all_gamma_holes_df["Post_Step_Position_mm"]

In [ ]:
# plot the distribution of the field to get a sense for the overall field values before analyzing the field in detail
for iteration, thresholdIN in zip(df.keys(), [93.6167, 101.242, 220.685]):
    length = len(df[iteration]["E_mag"])
    plt.hist(df[iteration]["E_mag"][df[iteration]["E_mag"]>0],bins=np.logspace(0,8,100),alpha=0.5,label=f"{iteration}: total leaves ={length}")
    plt.axvline(x=thresholdIN*1e3, linestyle=":")
#plt.hist(df2[df2["E_mag"]>0]["E_mag"],bins=np.logspace(-10,8,100),alpha=0.2,label="Iteration 78")
#plt.axvline(x=3e2,color="k")
plt.xscale("log")
plt.yscale("log")
plt.xlabel("|E| (V/m)")
plt.ylabel("Counts")
plt.title("Temperature: 425 K")
plt.legend()
plt.show()

In [ ]:
# --- 1. Scene Initialization and Data Loading ---

# Retrieve the field data dictionary for the current iteration (assumes 'df' is a list/dict)
fieldIN = df[iteration]
# Define the threshold for filtering electric field magnitude (E_mag)
threshold = 1e3 
# Set the maximum number of arrows to plot to maintain performance
max_arrows = 5000

# Initialize the 3D scene by plotting the geometry (e.g., detector structure)
# 'stacked_spheres' is the geometry to plot.
# 'edge_color' is set to black with an alpha (transparency) value of 350 (out of 511 max for trimesh visual.face_colors/edge_colors)
scene = plot_trimesh_edges_only(stacked_spheres, edge_color=[0, 0, 0, 350]) 

## ----------------------------------------------------
## --- 2. Filtering and Sampling High-Magnitude Points ---
## ----------------------------------------------------

# --- Filtering ---
# Create a boolean mask for all points where E_mag exceeds the threshold
large_magnitude_mask = fieldIN['E_mag'] > threshold

# Apply the mask to all three arrays ('pos', 'E', 'E_mag') simultaneously
# to create a new dictionary containing only the high-field points.
field_largevalues_masked = {
    'pos': fieldIN['pos'][large_magnitude_mask],
    'E': fieldIN['E'][large_magnitude_mask],
    'E_mag': fieldIN['E_mag'][large_magnitude_mask]
}

# Get the count of data points after initial filtering
N_large = len(field_largevalues_masked['E_mag'])

# --- Sampling ---
if N_large > max_arrows:
    
    # Randomly select a subset of indices to stay below the 'max_arrows' limit.
    # np.random.choice is the NumPy equivalent of a DataFrame's .sample() method.
    np.random.seed(42) # Set seed for reproducible sampling
    sample_indices = np.random.choice(
        N_large,         # Range of indices to choose from (0 to N_large - 1)
        size=max_arrows, # The number of indices to select
        replace=False    # Ensure each index is chosen only once
    )
    
    # Apply the sample indices to all arrays to create the final data dictionary for plotting
    field_plot = {
        'pos': field_largevalues_masked['pos'][sample_indices],
        'E': field_largevalues_masked['E'][sample_indices],
        'E_mag': field_largevalues_masked['E_mag'][sample_indices]
    }
    
    print(f"Sampled down to {max_arrows} points from {N_large} large-magnitude points.")

else:
    # If the filtered data is small enough, use it directly without sampling
    field_plot = field_largevalues_masked
    print(f"Used all {N_large} large-magnitude points for plotting.")


## ----------------------------------------------------
## --- 3. Normalization and Coloring Setup ---
## ----------------------------------------------------

# Get the vector field directions and magnitudes for the sampled points
directions = field_plot["E"]

# Calculate unit vectors (normalized directions)
# Use np.where to prevent division by zero if E_mag is exactly zero (though it shouldn't be after filtering)
directions_unit = np.where(
    field_plot["E_mag"][:, None] > 0, 
    directions / field_plot["E_mag"][:, None], 
    0
)

# Use Logarithmic scaling for magnitude visualization
# Add a small epsilon (1e-12) before log to avoid log(0), which results in -inf
log_magnitudes = np.log10(fieldIN['E_mag'][fieldIN['E_mag'] > 1e4]) 

# Normalize log-magnitudes to the range [0, 1] for colormapping
# np.ptp (peak-to-peak) is range (max - min)
min_log = log_magnitudes.min()
ptp_log = np.ptp(log_magnitudes)

# The normalization uses the min/ptp of the *sampled* data, not the full dataset
# A small epsilon is added to the divisor to prevent division by zero in case ptp is 0
log_magnitudes = np.log10(field_plot['E_mag']) 
norm_magnitudes = (log_magnitudes - min_log) / (ptp_log + 1e-12)

print(f"Log(E_mag) Min: {min_log}, Range (PtP): {ptp_log}")

# Choose a Colormap (e.g., 'jet')
cmap = plt.cm.jet
# Map the normalized magnitudes (0 to 1) to colors (RGBA floats 0.0 to 1.0)
colors_rgba = cmap(norm_magnitudes)

# Convert the RGBA colors from floats (0.0-1.0) to 8-bit integers (0-255) for trimesh
colors_rgba = (colors_rgba * 255).astype(np.uint8)


## ----------------------------------------------------
## --- 4. Arrow Creation and Visualization ---
## ----------------------------------------------------

# Define base dimensions for the visualization
base_arrow_length = 0.05  # Base length before scaling by magnitude
arrow_radius = 0.001       # Radius of the arrow shaft
cone_ratio = 0.2           # Ratio of the cone length to the total arrow length

# Scale the base length by the normalized magnitude for visual encoding
scaling_factor = 0.5  # Overall factor to control arrow visibility
scaled_lengths = base_arrow_length * scaling_factor * norm_magnitudes

# Iterate over each sampled point to create and place an arrow
for pos, dir_vec, color, magnitude_norm, scaled_length in zip(
    field_plot["pos"], 
    directions_unit, 
    colors_rgba, 
    norm_magnitudes,
    scaled_lengths
):
    # Skip if the direction vector has zero magnitude (E_mag was not filtered perfectly or is near zero)
    if np.linalg.norm(dir_vec) == 0:
        continue

    # Calculate arrow dimensions
    arrow_length = scaled_length
    cone_length = arrow_length * cone_ratio
    shaft_length = arrow_length - cone_length

    # Create Arrow Geometry in trimesh (built along the Z-axis by default)

    # 1. Create shaft (cylinder)
    shaft = trimesh.creation.cylinder(radius=arrow_radius, height=shaft_length, sections=12)
    # Move the shaft so its base is at z=0
    shaft.apply_translation([0, 0, shaft_length / 2]) 

    # 2. Create cone (cone)
    cone = trimesh.creation.cone(radius=arrow_radius * 2, height=cone_length, sections=12)
    # Position the cone base to touch the top of the shaft
    cone.apply_translation([0, 0, shaft_length])

    # 3. Combine parts into a single trimesh object
    arrow = trimesh.util.concatenate([shaft, cone])

    # Set the computed color (scaled by magnitude) for all faces of the arrow
    arrow.visual.face_colors = np.tile(color, (arrow.faces.shape[0], 1))

    # Calculate the necessary transformation to place and orient the arrow

    # a. Compute rotation matrix to align the default Z-axis ([0, 0, 1]) to the direction vector (dir_vec)
    transform = trimesh.geometry.align_vectors([0, 0, 1], dir_vec)
    
    # b. Set the translation part of the transformation matrix (the arrow's position)
    transform[:3, 3] = pos
    
    # c. Apply the full rotation and translation
    arrow.apply_transform(transform)

    # Add the colored and positioned arrow to the visualization scene
    scene.add_geometry(arrow)

# # # Target point
# #location = np.array([[-0.1, 0, 0.1 - 0.015 + 0.037]])
# location = np.array([[0, -0.1, 0.1]]) 
# #location = np.array([[0, 0, 0.2- 0.015]])   
 
# radius_mm = 10/1000 # units: um
# positions = df[iteration]["pos"]
# efield = df[1]["E"]
# emag = df[1]["E_mag"]

# # Vectorized distance computation (distance between positions and target)
# distances = np.linalg.norm(positions - location, axis=1)

# # 1. Get indices of all points within the specified radius
# # The result is a boolean array
# within_radius_mask = distances <= radius_mm

# # Get the actual indices
# indices_within_radius = np.where(within_radius_mask)[0]

# # Count the points found
# count = len(indices_within_radius)
# position_points = positions[indices_within_radius]
# #E_points = efield[indices_within_radius]
# #E_mag_points = emag[indices_within_radius]
# emag_values = emag[indices_within_radius]

# print(count,np.mean(emag_values[emag_values>0]))

# # Create small spheres at each point
# spheres = []
# for point in position_points:
#     sphere = trimesh.creation.icosphere(radius=0.001)  # adjust radius for point size
#     sphere.apply_translation(point)
#     sphere.visual.face_colors = [255, 0, 0, 255]  # red spheres
#     spheres.append(sphere)

# # Combine all meshes into a scene
# scene.add_geometry(spheres)

# Display the final scene containing the geometry and the vector field arrows
scene.show()

### read in processed fieldmaps:

Processed fieldmaps have a set spherical radius around the point of interest (goal: replicate Fig. 7 from Zimmerman et al., 2016)

In [ ]:
target_point = np.array([-0.1, 0, 0.1 - 0.015 + 0.037])

configIN = "onlyphotoemission"
directory = "../build-dissipationRefinedGrid-initial8max0.8final12-500000"

filenames = sorted(glob.glob(f"{directory}/fieldmaps/*{configIN}*.txt")) #{iteration}
print(filenames)

Evector_atpoint_iterations = []

df  = read_data_format_efficient(filenames,scaling=True)

for keyIN in df.keys():

    print(keyIN)

    # Extract data
    points = df[keyIN]["pos"] 
    vectors = df[keyIN]["E"]
    magnitudes = df[keyIN]["E_mag"]

    # ----------------------------------------------------
    # Step 2: Build KDTree and find nearest neighbor
    # ----------------------------------------------------
    tree = cKDTree(points)
    dist, idx = tree.query(target_point)

    # Nearest neighbor field
    E_vec_at_point = vectors[idx]
    E_mag_at_point = magnitudes[idx]

    # Store results
    Evector_atpoint_iterations.append((keyIN, E_vec_at_point, E_mag_at_point))

del df

In [ ]:
Evector_atpoint_iterations

In [ ]:
directory = "/storage/scratch1/5/avira7/Grain-Charging-Simulation-Data/stacked-sphere/output110525/processed-fieldmaps"
processed_data = load_h5_to_dict(f"processed-fieldmaps/PE_425K_initial8max0.8final12_sphere50um.h5")
# Retrieve the field data dictionary for the current iteration (assumes 'df' is a list/dict)

# takes ~2 minutes to read in 12 GB 

In [ ]:
import numpy as np
from scipy.spatial import cKDTree

# Target point
target_point = np.array([-0.1, 0, 0.1 - 0.015 + 0.037])
#target_point = np.array([-0.1-0.007, 0, 0.1 - 0.015 + 0.037])

Evector_atpoint_iterations = []

for keyIN in processed_data.keys():
    
    # Extract data
    points = processed_data[keyIN]["pos"] 
    vectors = processed_data[keyIN]["E"]
    magnitudes = processed_data[keyIN]["E_mag"]

    # ----------------------------------------------------
    # Step 2: Build KDTree and find nearest neighbor
    # ----------------------------------------------------
    tree = cKDTree(points)
    dist, idx = tree.query(target_point)

    # Nearest neighbor field
    E_vec_at_point = vectors[idx]
    E_mag_at_point = magnitudes[idx]

    #print(f"{keyIN}: E vector (nearest neighbor) = {E_vec_at_point}, |E| = {E_mag_at_point}")

    # Store results
    Evector_atpoint_iterations.append((int(keyIN.split("_")[1]), E_vec_at_point, E_mag_at_point))

In [ ]:
# Convert list of tuples to arrays for easier plotting
iterations, E_vectors, E_mag = zip(*Evector_atpoint_iterations)

# Plot
plt.figure(figsize=(8, 5))
plt.loglog(iterations, abs(np.array(E_vectors)[:, 0]), marker='.', linestyle='-', color='red') #abs(np.array(E_vectors)[:, 2])
plt.xlabel("Iteration")
plt.grid(True)
plt.show()

In [ ]:
# Retrieve the field data dictionary for the current iteration (assumes 'df' is a list/dict)
fieldIN = processed_data["iter_1"] 

plt.hist(fieldIN["E_mag"][fieldIN["E_mag"]>0],bins=np.logspace(0,8,100),alpha=0.5)
plt.xscale("log")
plt.yscale("log")
plt.xlabel("|E| (V/m)")
plt.ylabel("Counts")
plt.title("Temperature: 425 K")
plt.show()

In [ ]:
#  Varying density along a streamline
plt.streamplot(processed_data["iter_1"]["pos"][:,0], processed_data["iter_1"]["pos"][:,1], \
               processed_data["iter_1"]["E"][:,0], processed_data["iter_1"]["E"][:,1], density=[0.5, 1])

## Case 1: SW electrons and ions

### plot the field over each iteration:

In [ ]:
configIN = "onlysolarwind"

# --- Configurations ---
folder_path = [ "../build-temp600K-dynamicThreshold","../build-temp425K-dynamicThreshold", \
               "../build-425K-nodissipation-initial6-0.8Max-final9", "../build-425K-withoutdissipation"]
temperatures = [600,425,425,425]
notes = ["initial6max0.2final9(dissipation)","initial6max0.2final9(dissipation)","initial6max0.7final9(noDissipation)","initial5max0.8final9(noDissipation)"]

# Target point (Fixed for all configurations)
location = np.array([-0.1, 0, 0.1 - 0.015 + 0.037]) 

# --- Parallel Processing Worker Function ---

def process_config(tempIN, folderIN, noteIN, configIN, location):
    """
    Worker function to process a single configuration. 
    Returns the unique key and the calculated results.
    """
    key_name = f"SW_{tempIN}K_{noteIN}"
    print(f"--- Processing {key_name} in {folderIN} (Worker Process) ---\n")

    # 1. Read Field Data
    filenames = sorted(glob.glob(f"{folderIN}/fieldmaps/*{configIN}*.txt"))
    fields_SW = read_data_format_efficient(filenames, scaling=True) 
    first_key = list(fields_SW.keys())[0]

    # Prepare dictionary for this single result
    config_results: Dict[str, Any] = {}
    
    # 2. Compute and Store Field at Target Location
    # The function now returns a dict {'iter', 'E', 'E_mag'} for all iterations
    config_results["fieldAtTarget"] = compute_nearest_field_vector(fields_SW, target=location, start=first_key)
    
    # 3. Compute and Store the list of leaf lengths (number of points per iteration)
    config_results["lengthLeaves"] = [len(fields_SW[keyIN]["pos"]) for keyIN in fields_SW.keys()]
    config_results["gradRefinements"] = [fields_SW[keyIN]["gradRefinements"] for keyIN in fields_SW.keys()]
    
    # If the fields_PE dictionary is very large, deleting it immediately frees memory
    del fields_SW 
    
    # Return the key and the results to the main thread
    return key_name, config_results

# MASTER DICTIONARY to store all results securely
results_data: Dict[str, Dict[str, Any]] = {}

# --- Parallel Processing Loop ---

# Prepare the list of arguments for the executor
configs = zip(temperatures, folder_path, notes)
args_list = [(temp, folder, note, configIN, location) for temp, folder, note in configs]

MAX_WORKERS = len(temperatures) # Use one worker per configuration

print(f"--- Starting Parallel Processing with {MAX_WORKERS} workers ---")

# Use ProcessPoolExecutor for CPU-bound tasks
with concurrent.futures.ProcessPoolExecutor(max_workers=MAX_WORKERS) as executor:
    
    # Submit all tasks and store the future objects
    futures = [executor.submit(process_config, *args) for args in args_list]
    
    # Collect results as they complete
    for future in concurrent.futures.as_completed(futures):
        try:
            key, data = future.result()
            results_data[key] = data
        except Exception as exc:
            print(f'Configuration generated an exception: {exc}')

print(f"--- Completed Parallel Processing ---")

# takes 2 minutes to run

In [ ]:
# --- Plotting and Analysis ---

print("\n--- Generating Plot for SW Comparison ---")

# 1. Load comparison data from Zimmerman
zimmerman_SWdata = pd.read_csv("Fig7a-SW.csv")
zimmerman_PEdata = pd.read_csv("Fig7a-PE.csv")
zimmerman_PEandSWdata = pd.read_csv("Fig7a-PE+SW.csv")

# 2. Define constants and calculate conversion factor
# Simulation world area size in m^2 (600 um x 600 um)
WORLD_XY_AREA_SQ_M = 600 * 600 / (1e6**2) 

# Number of particles (protons) injected into the active area per iteration
PARTICLES_PER_ITERATION = 160440

# Ion flux calculated from the simulation area (ions/m^2)
FLUX_PER_ITERATION = PARTICLES_PER_ITERATION / WORLD_XY_AREA_SQ_M 

# Photoemission (PE) ion flux value (e/m^2/s). This factor combines 
# the effective current (4 uA/cm^2) and conversion to e/m^2/s.
SW_ION_FLUX = 3e-7 * 6.241509e18 

# Conversion factor: Time (s) per simulation iteration
CONVERT_ITERATION_PE_TIME = FLUX_PER_ITERATION / SW_ION_FLUX
print(f"Conversion Factor (s/iteration): {CONVERT_ITERATION_PE_TIME:.3e}")

# 3. Define plot parameters (Colors)
# Choose a continuous colormap and generate discrete colors based on the number of temperatures
CMAP_NAME = 'jet' 
discrete_cmap = plt.get_cmap(CMAP_NAME, len(temperatures))
color_list_rgba = [discrete_cmap(i) for i in np.linspace(0, 1, len(temperatures))]

# 4. Generate Plot (Log-Log)
plt.figure()

# Plot reference data (Zimmerman)
plt.plot(10**zimmerman_SWdata["x"], zimmerman_SWdata[" y"], '--', label="SW (Zimmerman 2016)", color="r", lw=4)
plt.plot(10**zimmerman_PEdata["x"], zimmerman_PEdata[" y"], 'g:', label="PE (Zimmerman 2016)")
plt.loglog(10**zimmerman_PEandSWdata["x"], zimmerman_PEandSWdata[" y"], 'b:', label="PE+SW (Zimmerman 2016)")

# Plot simulation results
for tempIN, colorIN, noteIN in zip(temperatures, color_list_rgba, notes):    
    key = f"SW_{tempIN}K_{noteIN}"
    
    # Plot E-field magnitude at target location
    plt.plot((results_data[key]["fieldAtTarget"]["iter"] -0.5)* CONVERT_ITERATION_PE_TIME, 
             results_data[key]["fieldAtTarget"]["E"][:,0], #results_data[key]["fieldAtTarget"]["E_mag"] 
             '.-',
             label=f"{tempIN} K: {noteIN}",
             lw=0.5, 
             color=colorIN)

plt.xlabel("Time [s]")
# Plotting the E-field magnitude (|E|)
plt.ylabel(r"$|E_x$| (V/m)") 
plt.legend()
plt.grid(True)

# Set axes limits
plt.ylim(4.8e3, 2.8e5)
plt.xlim(7.4e-2, 1.25e1)

plt.tight_layout()
plt.show()

In [ ]:
# Plot simulation results
for tempIN, colorIN, noteIN in zip(temperatures, color_list_rgba, notes):    
    key = f"SW_{tempIN}K_{noteIN}"
    
    # Plot E-field magnitude at target location
    plt.semilogy(results_data[key]["fieldAtTarget"]["iter"],
             np.array(results_data[key]["lengthLeaves"])/1e6,
             '.-',
             label=f"{tempIN} K: {noteIN}",
             lw=0.5, 
             color=colorIN)
plt.xlabel("Iteration #")
plt.ylabel("# of Total Leaf Nodes (millions)")
plt.axhline(y=1, color='k',lw=1)
plt.title("SW Case")
plt.legend()
plt.show()

In [ ]:
# Plot simulation results
for tempIN, colorIN, noteIN in zip(temperatures, color_list_rgba, notes):    
    key = f"SW_{tempIN}K_{noteIN}"
    
    # Plot E-field magnitude at target location
    plt.semilogy(results_data[key]["fieldAtTarget"]["iter"],
             np.array(results_data[key]["gradRefinements"])/1e6,
             '.-',
             label=f"{tempIN} K: {noteIN}",
             lw=0.5, 
             color=colorIN)
plt.xlabel("Iteration #")
plt.ylabel("# of Gradient Refinements (millions)")
plt.title("SW Case")
plt.axhline(y=1, color='k',lw=1)
plt.legend()
plt.show()

In [ ]:
## test one file ##

# configIN = "onlysolarwind"
# directory_path = "../build-leakage/" # takes 12 minutes to read in with this data
# #directory_path = "../build-adaptive-barns-fixed/"

# filenames = sorted(glob.glob(f"{directory_path}/fieldmaps/*{configIN}*.txt"))
# fields_SW = read_data_format_efficient(filenames,scaling=True)   
# 
# # Target point
# location = np.array([-0.1, 0, 0.1-0.015+0.037]) 
# # return the electric field at that location
# Efield_SW_location = compute_nearest_field_vector(fields_SW, target=location, start=1)     

### check the minimum distance between point in field map 

In [ ]:
## test one file ##

directory_path = "../build-425K-nodissipation-initial6-0.8Max-final9/"
filenames = sorted(glob.glob(f"{directory_path}/fieldmaps/*{configIN}*.txt"))
fields_SW = read_data_format_efficient(filenames,scaling=True) 

In [ ]:
# Assuming 'fields_PE' is your list/dictionary structure and 
# fields_PE[1]['pos'] is a NumPy array of shape (N, 3), where N is the number of points.
# Example data (replace this with your actual data):
data_points = fields_SW[1]['pos'] 
# data_points = np.array([
#     [1.0, 1.0, 1.0],
#     [1.001, 1.0, 1.0],  # Very close point
#     [2.0, 2.0, 2.0],
#     [5.0, 5.0, 5.0]
# ], dtype=np.float32)

# 1. Build the KD-Tree
# This organizes the points in a spatial structure for efficient nearest neighbor search.
tree = cKDTree(data_points)

# 2. Query for the 2 nearest neighbors of every point
# The 'k=2' parameter tells the query to find the distance to the 2 nearest neighbors:
# - The 1st neighbor (k=1) is always the point itself (distance = 0.0).
# - The 2nd neighbor (k=2) is the closest *other* point.
distances, indices = tree.query(data_points, k=2)

# 3. Extract the minimum non-zero distance
# The minimum distance between any unique pair of points is the minimum value 
# in the array of distances to the second nearest neighbor (distances[:, 1]).
min_distance = np.min(distances[:, 1])*1000

print(f"The total number of points (voxels) is: {len(data_points)}")
print(f"The minimum distance between any two unique voxels is: {min_distance} um")

# takes around ~10 seconds to run


### calculate # of iterations for direct comparison with Zimmerman:

In [ ]:
directory_path = "../build-wang-comparison/root/"
#directory_path = "/storage/scratch1/5/avira7/Grain-Charging-Simulation-Data/build-dissipationRefinedGrid-initial8max0.8final12/root"
configIN = "solarwind"
filelist = sorted(glob.glob(f"{directory_path}/*iteration*{configIN}*num50000.root"))

all_incident_protons_inside, all_incident_electrons_inside = [],[]

for fileIN in filelist:

    print(fileIN.split("/")[-1])
    number_str = fileIN.split("/")[-1].split("_")[1]
    iterationNUM = int(''.join(filter(str.isdigit, number_str)))

    # read data from different iterations
    vars()["protons_inside_"+str(number_str)], vars()["electrons_inside_"+str(number_str)] = calculate_stats(read_rootfile(fileIN.split("/")[-1], directory_path=directory_path), 
                                                                                                             config=configIN)
    all_incident_protons_inside.append(vars()["protons_inside_"+str(number_str)])
    all_incident_electrons_inside.append(vars()["electrons_inside_"+str(number_str)])
    print(78*"-")

    break

In [ ]:
# Concatenate all iterations into single DataFrames
all_incident_protons_inside_df = pd.concat(all_incident_protons_inside, ignore_index=True)
all_incident_electrons_inside_df = pd.concat(all_incident_electrons_inside, ignore_index=True)

In [ ]:
surf, ilm_values,_ = plot_face_illumination(electrons_inside_iteration0, stacked_spheres, vmin=0, vmax=10)
# Make sure each triangle has its own unique vertices
surface_edited = surf.copy()
surface_edited.unmerge_vertices()
surface_edited.visual.vertex_colors = None
surface_edited.show()

In [ ]:
protons_inside_iteration50

In [ ]:
plt.hist(ilm_values, bins=np.logspace(-1,3,100))
plt.xscale("log")
plt.title("# of Photons hitting each Voxel")
plt.show()

print(f"Distribution Stats: mean {np.mean(ilm_values[ilm_values!=0])}, median {np.median(ilm_values[ilm_values!=0])}, max {np.max(ilm_values)}")

# our area is a factor of 4 smaller than their area 
#print(f"for one iteration, mean # of particles in each equivalently sized voxel is {np.mean(ilm_values[ilm_values!=0])}") <- no longer needed, made voxels similar sizes
voxel_area = 0.0004/(1000)**2 # rough area approximated from python (0.4 micron2)
zimmerman_charge = 1*(1e-6) # C/m2
zimmerman_electronnum = (zimmerman_charge/1.60217663e-19)*voxel_area
print(f"will take {zimmerman_electronnum/(np.mean(ilm_values[ilm_values!=0]))} iterations at this rate to get to the photoemission flux ranges shown at 3 seconds")

### make a movie of all surface potential for each iteration:

In [ ]:
directory = "figures/solarwind/"

for num in range(0, iterationNUM+1):
    surface, _, facecolors = plot_surface_potential_fornegativepositive_charge(
        vars()["electrons_inside_stackediteration"+str(num)], 
        vars()["protons_inside_stackediteration"+str(num)], 
        stacked_spheres, 
        vmin=-1.8, vmax=1.8
    )

    surface_edited = surface.copy()
    surface_edited.unmerge_vertices()  # Ensure unique vertices per face

    # Crop bounding box
    bbox_min = np.array([-0.2, -0.3, 0])
    bbox_max = np.array([ 0.2,  0.1, 100])

    in_box = np.all((surface_edited.vertices >= bbox_min) & 
                    (surface_edited.vertices <= bbox_max), axis=1)

    face_mask = np.all(in_box[surface_edited.faces], axis=1)

    cropped = surface_edited.submesh([face_mask], only_watertight=False, append=True)
    cropped_colors = facecolors[face_mask]/255  # Crop facecolors to match cropped mesh

    # Plotting
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, projection='3d')

    mesh = Poly3DCollection(cropped.triangles, alpha=1.0)
    mesh.set_facecolor(cropped_colors)  # Apply correct colors
    mesh.set_edgecolor('k')          # edge color
    mesh.set_linewidths(0.1)         # edge line width

    ax.add_collection3d(mesh)
    ax.set_title(f"Iteration {num}")
    ax.title.set_position((0.5, 0.1))  # manually control title position

    # Scale
    scale = surface_edited.bounds.flatten()
    ax.auto_scale_xyz(scale, scale, scale)

    ax.set_axis_off()

    filename = f"{directory}iteration_{num}.png"
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close(fig)
    if num%10==0:
        print(f"Saved: {filename}")

print(f"All plots saved to {directory}")


In [ ]:
directory = "figures/solarwind/"

for num in range(0, iterationNUM + 1):
    # Get face illumination for electrons and protons
    surface_e, _, facecolors_e = plot_face_illumination(
        vars()["electrons_inside_stackediteration" + str(num)],
        stacked_spheres, vmin=0, vmax=100
    )

    surface_p, _, facecolors_p = plot_face_illumination(
        vars()["protons_inside_stackediteration" + str(num)],
        stacked_spheres, vmin=0, vmax=100
    )

    def crop_and_prepare(surface, facecolors):
        surface = surface.copy()
        surface.unmerge_vertices()

        # Crop bounding box
        bbox_min = np.array([-0.2, -0.3, 0])
        bbox_max = np.array([ 0.2,  0.1, 100])

        in_box = np.all((surface.vertices >= bbox_min) & 
                        (surface.vertices <= bbox_max), axis=1)

        face_mask = np.all(in_box[surface.faces], axis=1)
        cropped = surface.submesh([face_mask], only_watertight=False, append=True)
        cropped_colors = facecolors[face_mask] / 255.0
        return cropped, cropped_colors

    cropped_e, colors_e = crop_and_prepare(surface_e, facecolors_e)
    cropped_p, colors_p = crop_and_prepare(surface_p, facecolors_p)

    # Plotting both side by side
    fig = plt.figure(figsize=(14, 7))

    for i, (cropped, colors, title) in enumerate([
        (cropped_e, colors_e, "Electron Illumination"),
        (cropped_p, colors_p, "Proton Illumination")
    ]):
        ax = fig.add_subplot(1, 2, i + 1, projection='3d')
        mesh = Poly3DCollection(cropped.triangles, alpha=1.0)
        mesh.set_facecolor(colors)
        mesh.set_edgecolor('k')
        mesh.set_linewidths(0.1)
        ax.add_collection3d(mesh)

        scale = cropped.bounds.flatten()
        ax.auto_scale_xyz(scale, scale, scale)
        ax.set_axis_off()
        ax.set_title(f"{title}\nIteration {num}", pad=5)

    filename = f"{directory}iteration_{num}.png"
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close(fig)

    if num % 10 == 0:
        print(f"Saved: {filename}")
        
print(f"✅ All plots saved to {directory}")


In [ ]:
iterationIN=0

directory = "figures/solarwind/"

for num in range(0,iterationNUM):
    surface,_ = plot_surface_potential_fornegativepositive_charge(vars()["electrons_inside_stackediteration"+str(num)], vars()["protons_inside_stackediteration"+str(num)], stacked_spheres, vmin=-10,vmax=10)

    # plt.hist(facolors[facolors!=0],bins=50)
    # plt.show()

    # Make sure each triangle has its own unique vertices
    surface_edited = surface.copy()
    surface_edited.unmerge_vertices()
    surface_edited.visual.vertex_colors = None
    surface_edited.show()

In [ ]:
# Define colormap and normalization
cmap = plt.cm.OrRd #seismic
norm = Normalize(vmin=0, vmax=100)

# Create a figure and a single axis for the colorbar
fig, ax = plt.subplots(figsize=(6, 1))
fig.subplots_adjust(bottom=0.5)

# Create the colorbar
cb = ColorbarBase(ax, cmap=cmap, norm=norm, orientation='horizontal')
cb.set_label('# of Particles / face')  # Optional label

plt.show()


In [ ]:
bbox_min = np.array([-0.25, -0.31, 0])
bbox_max = np.array([ 0.15,  0.1,  100])

# Filter vertices
in_box = np.all((surface_edited.vertices >= bbox_min) & 
                (surface_edited.vertices <= bbox_max), axis=1)

# Get face indices where all 3 vertices are in the box
face_mask = np.all(in_box[surface_edited.faces], axis=1)

# Extract submesh
cropped = surface_edited.submesh([face_mask], only_watertight=False, append=True)
cropped.show()

## Case 2: Photoemission (incident gammas)

### plot the field over each iteration:

In [ ]:
# directory = "/storage/scratch1/5/avira7/Grain-Charging-Simulation-Data/stacked-sphere/output111025/processed-fieldmaps"
# processedResults = load_h5_to_dict(f"{directory}/PE_425K_initial8max0.8final12_noDissipation_sphere50um-throughXX.h5")

In [ ]:
# --- configuration ---
directory = "/storage/scratch1/5/avira7/Grain-Charging-Simulation-Data/stacked-sphere/output111025/processed-fieldmaps"
h5_filenames = glob.glob(f"{directory}/*.h5")

# --- helper function for one key ---
def process_key(args):

    keyIN, val, target_point = args

    points = val["pos"]
    vectors = val["E"]
    magnitudes = val["E_mag"]

    # radius = 6e-3  # spherical averaging radius
    radius = 2.5e-3  # spherical averaging radius
    
    mask = np.sum((points - target_point)**2, axis=1) <= radius**2
    if not np.any(mask):
        # Return placeholders if no points in sphere
        return -1, np.full(3, np.nan), np.nan, np.full(3, np.nan), 0

    avg_position = points[mask].mean(axis=0)
    E_vec = vectors[mask].mean(axis=0)
    E_mag = magnitudes[mask].mean(axis=0)
    E_vec_errors = vectors[mask].std(axis=0) /np.sqrt(len(magnitudes[mask]))
    #point_err = np.abs(avg_position - target_point)
    return int(keyIN.split("_")[1]), E_vec, E_mag, E_vec_errors,len(magnitudes[mask]) 

    # tree = cKDTree(points)
    # dist, idx = tree.query(target_point)

    # # Nearest neighbor field
    # E_vec_at_point = vectors[idx]
    # E_mag_at_point = magnitudes[idx]

    # return int(keyIN.split("_")[1]), E_vec_at_point, E_mag_at_point, dist 

# --- extract metadata from filename ---
def parse_filename_metadata(filename):
    """
    Only processes files with:
      - Temperature: 425 K
      - pos_value: -0.1
    Example:
    PE_425K_initial8max0.8final12_RefinedGridDissipation_500000particles_Sphere20um_pos-0.1_through26.h5
    
    """

    base = os.path.basename(filename)

    case = base.split("_")[0]

    # Temperature
    temp_match = re.search(r'_(\d+)K_', base)
    temperature = int(temp_match.group(1)) if temp_match else np.nan

    # Position
    pos_match = re.search(r'_pos([-+]?\d*\.?\d+)_through', base)
    pos_value = float(pos_match.group(1)) if pos_match else np.nan

    # Sphere size (if any)
    sphere_match = re.search(r'_sphere(\d+)um', base, re.IGNORECASE)
    sphere_um = int(sphere_match.group(1)) if sphere_match else np.nan

    # Octree parameters: initial, grad threshold, final
    octree_match = re.search(r'_initial(\d+)max([-+]?\d*\.?\d+)final(\d+)', base)
    if octree_match:
        octree_params = {
            "initial_depth": int(octree_match.group(1)),
            "percent_gradThreshold": float(octree_match.group(2)),
            "final_depth": int(octree_match.group(3))
        }
    else:
        octree_params = {"initial_depth": np.nan, "grad_threshold": np.nan, "final_depth": np.nan}

    # # Number of particles (e.g., "_500000particles_")
    # particle_match = re.search(r'_(\d+)particles', base, re.IGNORECASE)
    # num_particles = int(particle_match.group(1)) if particle_match else 100000

    # # --- filtering condition ---
    # if not (temperature == 425 and np.isclose(pos_value, -0.1)) and (num_particles == 500000):
    #     # Skip file if criteria not met
    #     return None

    # Target point
    #target_point = np.array([pos_value - 0.0073, 0, 0.1 - 0.015 + 0.037 - 0.00073])
    target_point = np.array([pos_value, 0, 0.1 - 0.015 + 0.037])

    metadata = {
        "filename": base,
        "case": case,
        "temperature": temperature,
        "target_point": target_point,
        "sphere_um": sphere_um,
        "octree": octree_params,
       # "num_particles": num_particles
    }

    return metadata


# --- worker for one file ---
def process_file(fileIN):
    metadata = parse_filename_metadata(fileIN)
    
    # # Filter: skip files with 'through' files that have processed < 40 iterations
    # through_idx = int(re.search(r'_through(\d+)\.h5', fileIN).group(1))
    # if through_idx < 40:
    #     return None

    if metadata is None:
        return None

    print(f"→ Started {os.path.basename(fileIN)}\n", flush=True)
    processedResults = load_h5_to_dict(fileIN)
    key_prefix = os.path.basename(fileIN).split('_through')[0]

    keys = list(processedResults.keys())
    n_keys = len(keys)

    # --- inner parallelization across keys ---
    args_list = [(k, processedResults[k], metadata["target_point"]) for k in keys]

    with ThreadPoolExecutor(max_workers=4) as tpool:  # inner parallel threads
        results = list(tpool.map(process_key, args_list))

    ids, E_vecs, E_mags, standard_errors, N = zip(*results)
    ids = np.array(ids)
    E_vecs = np.array(E_vecs)
    E_mags = np.array(E_mags)
    standard_errors = np.array(standard_errors)
    num_points = np.array(N)

    print(f"✓ Finished {os.path.basename(fileIN)}", flush=True)
    return key_prefix, {
        "iter": ids,"E_vecs": E_vecs,"E_mags": E_mags,"point_errors": standard_errors, "N":num_points, "metadata": metadata
    }

# --- parallel execution across files ---
num_cores = 2
all_processed = {}

with ProcessPoolExecutor(max_workers=num_cores) as executor:
    futures = {executor.submit(process_file, f): f for f in h5_filenames}
    for fut in as_completed(futures):
        fileIN = futures[fut]
        try:
            result = fut.result()
            if result is not None:
                key_prefix, data = result
                all_processed[key_prefix] = data
                print(f"✔ Processed {os.path.basename(fileIN)}\n", flush=True)
        except Exception as e:
            print(f"❌ Error in {os.path.basename(fileIN)}: {e}\n", flush=True)


In [ ]:

from scipy.optimize import curve_fit
# Import interpolate for numerical method
from scipy.interpolate import CubicSpline
import matplotlib.gridspec as gridspec

# --- 1. CONFIGURATION AND DATA LOADING ---
print("\n--- Starting Data Processing and Plot Generation ---")

# Define Data Keys for Fitting and Subtraction
KEY_FIT = 'PE_425K_initial8max0.8final12_noDissipation_sphere50um_pos-0.1'
KEY_TARGET = 'PE_425K_initial8max0.8final12_RefinedGridDissipation_sphere20um_pos-0.1'

# Load external literature data
zimmerman_SWdata = pd.read_csv("literature-data/Fig7a-SW.csv")
zimmerman_PEdata = pd.read_csv("literature-data/Fig7a-PE.csv")
zimmerman_PEandSWdata = pd.read_csv("literature-data/Fig7a-PE+SW.csv")

# Simulation Parameters (used for time conversion)
WORLD_XY_AREA_SQ_M = 300 * 300 / (1e6**2) # World area (m^2)

# PE Conversion Factor
PARTICLES_PER_ITERATION_PE = 81775
FLUX_PER_ITERATION_PE = PARTICLES_PER_ITERATION_PE / WORLD_XY_AREA_SQ_M
PE_ION_FLUX = 4e-6 * 6.241509e18
CONVERT_ITERATION_PE_TIME = FLUX_PER_ITERATION_PE / PE_ION_FLUX
print(f"PE Conversion Factor (s/iteration): {CONVERT_ITERATION_PE_TIME:.3e}")

# SW Conversion Factor
PARTICLES_PER_ITERATION_SW = 30601
FLUX_PER_ITERATION_SW = PARTICLES_PER_ITERATION_SW / WORLD_XY_AREA_SQ_M
SW_ION_FLUX = 3e-7 * 6.241509e18
CONVERT_ITERATION_SW_TIME = FLUX_PER_ITERATION_SW / SW_ION_FLUX
print(f"SW Conversion Factor (s/iteration): {CONVERT_ITERATION_SW_TIME:.3e}")

# Color Map Setup
CMAP_NAME = 'Dark2'
discrete_cmap = plt.get_cmap(CMAP_NAME, len(all_processed.keys()) + 1)
color_list_rgba = [discrete_cmap(i) for i in np.linspace(0, 1, len(all_processed.keys()) + 1)]

# --- 2. CURVE FITTING AND EXTRAPOLATION ---

# Define the new fitting function (Polynomial of Order 3)
def poly_curve(t, a, b, c, d):
    """
    Function: a*t^3 + b*t^2 + c*t + d (Polynomial of Order 3)
    """
    return a*t**3 + b*t**2 + c*t + d


# Prepare data for fitting (PE_425K)
data_fit = all_processed[KEY_FIT]
x_fit = np.array(data_fit["iter"] - 1) * CONVERT_ITERATION_PE_TIME
y_fit = abs(data_fit["E_vecs"][:, 0])
y_fit_errors = data_fit["point_errors"][:, 0] # Get errors for KEY_FIT (50um)

# Prepare Target Data (PE_600K range)
data_target = all_processed[KEY_TARGET]
x_extrapolate = np.array(data_target["iter"] - 1) * CONVERT_ITERATION_PE_TIME
y_target = abs(data_target["E_vecs"][:, 0])
y_target_errors = data_target["point_errors"][:, 0] # Get errors for KEY_TARGET (20um)

# --- Choose Fitting/Extrapolation Method ---

# Method 1: Basic Polynomial Fit (Order 3) - CURRENT DEFAULT
method_label = f"Fit of {KEY_FIT.split('_')[1]} (Poly Order 3)"
try:
    # Using y_fit_errors as sigma for weighted fit
    popt, pcov = curve_fit(poly_curve, x_fit, y_fit, 
                           p0=[0, 0, 0, 1e4], sigma=y_fit_errors, absolute_sigma=True) # Added sigma
    
    A_fit, B_fit, C_fit, D_fit = popt
    print(f"\nFit Parameters for {KEY_FIT.split('_')[1]} (Poly): a={A_fit:.2e}, b={B_fit:.2e}, c={C_fit:.2e}, d={D_fit:.2e}")

    # Extrapolate and Calculate Subtraction
    y_extrapolated = poly_curve(x_extrapolate, *popt)
    y_subtraction = y_extrapolated - y_target

    # Estimate error in extrapolated fit for error propagation
    perr = np.sqrt(np.diag(pcov))
    # NOTE: J calculation assumes the x_extrapolate points are the basis for the estimated error
    J = np.array([3*x_extrapolate**2, 2*x_extrapolate, np.ones_like(x_extrapolate), np.zeros_like(x_extrapolate)]).T
    y_extrapolated_errors = np.sqrt(np.diag(J @ pcov @ J.T))


except RuntimeError:
    print("\n⚠️ Warning: Curve fitting failed. Check initial guess (p0) or fitting range.")
    y_extrapolated = np.zeros_like(x_fit)
    y_subtraction = np.zeros_like(x_fit)
    x_extrapolate = x_fit
    method_label = f"Fit of {KEY_FIT.split('_')[1]} (Failed, showing Zeros)"
    y_extrapolated_errors = np.zeros_like(x_extrapolate) # Set errors to zero if fit fails


# --- Calculate Percent Difference and its Error ---
y_denominator = np.where(y_extrapolated == 0, 1e-10, y_extrapolated)
y_percent_diff = (y_subtraction / y_denominator) * 100

# Error propagation for the difference: sqrt(error_fit^2 + error_target^2)
# Direct assignment works because y_target_errors has the same length as x_extrapolate
y_target_errors_interp = y_target_errors 

y_subtraction_errors = np.sqrt(y_extrapolated_errors**2 + y_target_errors_interp**2)

# Error propagation for the percentage
y_percent_diff_errors = (y_subtraction_errors / np.abs(y_denominator)) * 100 
# ---------------------------------------------------


# --- 3. PLOTTING SETUP (MAIN + SUBPLOT) ---

# Set up figure and grid layout (5:1 height ratio for main plot vs. residual plot)
fig = plt.figure(figsize=(8.01, 3.22))
gs = gridspec.GridSpec(2, 1, hspace=0.09, height_ratios=[5, 1])

# Main Plot (Top)
ax_main = fig.add_subplot(gs[0])
# Subtraction Plot (Bottom), sharing the x-axis
ax_sub = fig.add_subplot(gs[1], sharex=ax_main)

# --- 4. MAIN PLOT GENERATION (ax_main) ---

# Plot reference data (Zimmerman)
ax_main.plot(10**zimmerman_SWdata["x"], zimmerman_SWdata[" y"], '-', color="k", lw=3, alpha=0.3, label="Zimmerman SW/PE/PE+SW Ref.")
ax_main.plot(10**zimmerman_PEdata["x"], zimmerman_PEdata[" y"], '-', color="k", lw=3, alpha=0.3)
ax_main.plot(10**zimmerman_PEandSWdata["x"], zimmerman_PEandSWdata[" y"], '--', color="k", lw=3, alpha=0.3)

# Plot simulation data
color_list = [color_list_rgba[2],color_list_rgba[3],color_list_rgba[6]]
i=0

# Plot the fitted/extrapolated curve
ax_main.plot(x_extrapolate, y_extrapolated, '-', color=color_list[-1], lw=1.5, 
             label=f"{method_label} (Extrapolated)")

for keyIN, colorIN in zip(all_processed.keys(), color_list_rgba):
    
    # Define plotting variables outside of loop to use them later
    case = all_processed[keyIN]["metadata"]["case"]
    factor = CONVERT_ITERATION_PE_TIME if case == "PE" else CONVERT_ITERATION_SW_TIME
    tempIN = all_processed[keyIN]["metadata"]["temperature"]
    targetIN = all_processed[keyIN]["metadata"]["target_point"]
    
    x_data = np.array(all_processed[keyIN]["iter"] - 1) * factor
    y_data = abs(all_processed[keyIN]["E_vecs"][:, 0])
    y_err = all_processed[keyIN]["point_errors"][:, 0]
    
    tempIN = all_processed[keyIN]["metadata"]["temperature"]
    targetIN = all_processed[keyIN]["metadata"]["target_point"]
    # noteIN = keyIN.split("_")[2] # Not used in label for brevity

    # Filter plotting to only the relevant cases (e.g., specific position and T=425)
    if (targetIN[0] < 0) & (tempIN == 425) & ("Total" not in keyIN.split("_")[3]):
        
        print(keyIN)

        plot_color = color_list[i]

        # if keyIN == KEY_FIT:
        #     # Plot the data line
        #     ax_main.plot(x_data, y_data, '-', color=plot_color, lw=3)
        # else:
        # Plot the data line
        ax_main.plot(x_data, y_data, '-', color=plot_color, lw=1.5)
        
        # Use fill_between for the error region (Replaces errorbars)
        ax_main.fill_between(x_data, y_data - y_err, y_data + y_err, 
                            color=plot_color, alpha=0.15, 
                            label=None) # Set label=None to avoid extra legend entry
        i+=1

# Add original uncommented features back to ax_main
#ax_main.axvline(x=65 * CONVERT_ITERATION_PE_TIME, color='gray', linestyle='-.', lw=1, alpha=0.7, label="Vertical Marker")
ax_main.set_ylabel(r"$|E_x|$ (V/m)")
ax_main.ticklabel_format(axis='y', style='sci', scilimits=(0, 0))
#ax_main.set_yscale('log') # Use log scale for better visualization of power-law decay
ax_main.set_ylim(0,2e5) # Uncommented limits
# ax_main.set_xlim(7.4e-1, 1.25e1) # Uncommented limits

# Clean up main plot
# ax_main.grid(True, linestyle=':', alpha=0.5)
# ax_main.legend(loc='lower left', fontsize=8, ncol=2)
# Remove X-tick labels from the main plot
plt.setp(ax_main.get_xticklabels(), visible=False) 

# --- 5. SUBTRACTION PLOT GENERATION (ax_sub) ---

# PLOT PERCENT DIFFERENCE WITH SHADED ERROR REGION
ax_sub.plot(x_extrapolate, y_percent_diff, '-', color=color_list_rgba[3], lw=2,
            label=r"Relative Error: $\frac{|E_{Fit}| - |E_{Target}|}{|E_{Fit}|}$")
# Shaded region
# ax_sub.fill_between(x_extrapolate, y_percent_diff - y_percent_diff_errors, 
#                     y_percent_diff + y_percent_diff_errors, 
#                     color=color_list_rgba[2], alpha=0.2, label="Error Region")
ax_sub.set_xlabel("Time [s]")
#ax_sub.set_ylabel(r"% Diff")
# ax_sub.ticklabel_format(axis='y', style='sci', scilimits=(0, 0)) # Removed, % difference is typically not sci notation
# ax_sub.grid(True, linestyle=':', alpha=0.6)
# ax_sub.legend(loc='upper right', fontsize=8)
ax_sub.set_xlim(0,6) # Uncommented limit check (if sharing xlim works)
ax_sub.set_ylim(0,10) # Uncommented limit check (if sharing xlim works)
ax_sub.set_yticks([0,4,8])

# --- 6. SAVE AND SHOW ---
plt.savefig("figures/zimmerman_benchmark_summary.jpeg", bbox_inches="tight", dpi=300)
plt.show()

In [ ]:
iteration = 10
print(f"PE equivalent time for iteration#{iteration}: {(iteration-1)*CONVERT_ITERATION_PE_TIME}")

In [ ]:
discrete_cmap

In [ ]:
# --- Plotting and Analysis ---

print("\n--- Generating Plot for PE Comparison ---")

# 1. Load comparison data from Zimmerman
zimmerman_SWdata = pd.read_csv("literature-data/Fig7a-SW.csv")
zimmerman_PEdata = pd.read_csv("literature-data/Fig7a-PE.csv")
zimmerman_PEandSWdata = pd.read_csv("literature-data/Fig7a-PE+SW.csv")

# 2. Define constants and calculate conversion factor
# Simulation world area size in microns
WORLD_XY_AREA_SQ_M = 300 * 300 / (1e6**2) 

# Number of particles injected into the active area per iteration
PARTICLES_PER_ITERATION = 81775 #80559 # for the old geometry with smaller voxels #424975 # for 600by600 world

# Ion flux calculated from the simulation area (ions/m^2)
FLUX_PER_ITERATION = PARTICLES_PER_ITERATION / WORLD_XY_AREA_SQ_M 

# Photoemission (PE) ion flux value (e/m^2/s). This factor combines 
# the effective current (4 uA/cm^2) and conversion to e/m^2/s.
PE_ION_FLUX = 4e-6 * 6.241509e18 

# Conversion factor: Time (s) per simulation iteration
CONVERT_ITERATION_PE_TIME = FLUX_PER_ITERATION / PE_ION_FLUX
print(f"PE Conversion Factor (s/iteration): {CONVERT_ITERATION_PE_TIME:.3e}")


# Number of particles (protons) injected into the active area per iteration
PARTICLES_PER_ITERATION = 30601 #160440

# Ion flux calculated from the simulation area (ions/m^2)
FLUX_PER_ITERATION = PARTICLES_PER_ITERATION / WORLD_XY_AREA_SQ_M 

# Photoemission (PE) ion flux value (e/m^2/s). This factor combines 
# the effective current (4 uA/cm^2) and conversion to e/m^2/s.
SW_ION_FLUX = 3e-7 * 6.241509e18 

# Conversion factor: Time (s) per simulation iteration
CONVERT_ITERATION_SW_TIME = FLUX_PER_ITERATION / SW_ION_FLUX
print(f"SW Conversion Factor (s/iteration): {CONVERT_ITERATION_SW_TIME:.3e}")

# 3. Define plot parameters (Colors)
# Choose a continuous colormap and generate discrete colors based on the number of temperatures
CMAP_NAME = 'jet' 
discrete_cmap = plt.get_cmap(CMAP_NAME, len(all_processed.keys()) + 1)
color_list_rgba = [discrete_cmap(i) for i in np.linspace(0, 1, len(all_processed.keys()) + 1)]

# 4. Generate Plot (Log-Log)
plt.figure(figsize=(15,8))

# Plot reference data (Zimmerman)
plt.plot(10**zimmerman_SWdata["x"], zimmerman_SWdata[" y"], '--', color="r", lw=4) #, label="SW (Zimmerman 2016)"
plt.plot(10**zimmerman_PEdata["x"], zimmerman_PEdata[" y"], 'g:') #, label="PE (Zimmerman 2016)"
plt.plot(10**zimmerman_PEandSWdata["x"], zimmerman_PEandSWdata[" y"], 'b:') #, label="PE+SW (Zimmerman 2016)"

for keyIN, colorIN in zip(all_processed.keys(),color_list_rgba):
    if all_processed[keyIN]["metadata"]["case"] == "SW":
        FACTOR= CONVERT_ITERATION_SW_TIME
    else:
        FACTOR = CONVERT_ITERATION_PE_TIME

    tempIN = all_processed[keyIN]["metadata"]["temperature"]
    targetIN = all_processed[keyIN]["metadata"]["target_point"]
    noteIN = keyIN.split("_")[2]

    if targetIN[0] == 0.1:
        plt.plot(np.array(all_processed[keyIN]["iter"]) * FACTOR, \
                abs(all_processed[keyIN]["E_vecs"][:,0]), marker='.', linestyle='-', color=colorIN,lw=0.5,\
                label=f"{keyIN}") #{tempIN}: {noteIN}, pos: {targetIN}

plt.xlabel("Time [s]")
# Plotting the E-field magnitude (|E|)
plt.axvline(x=65*CONVERT_ITERATION_PE_TIME)
plt.ylabel(r"$|E_x$| (V/m)") 
#plt.ylabel(r"$|E$| (V/m)") 
plt.legend(bbox_to_anchor=(1,1))
plt.grid(True)

# Set axes limits
#plt.ylim(4.8e3, 2.8e5)
#plt.xlim(7.4e-1, 1.25e1)
plt.xlim(0,4)

plt.tight_layout()
plt.show()

In [ ]:
# Plot simulation results
for tempIN, colorIN, noteIN in zip(temperatures, color_list_rgba, notes):    
    key = f"PE_{tempIN}K_{noteIN}"
    
    # Plot E-field magnitude at target location
    plt.semilogy(results_data[key]["fieldAtTarget"]["iter"],
             np.array(results_data[key]["lengthLeaves"])/1e6,
             '.-',
             label=f"{tempIN} K: {noteIN}",
             lw=0.5, 
             color=colorIN)
plt.xlabel("Iteration #")
plt.ylabel("# of Total Leaf Nodes (millions)")
plt.title("PE Case")
plt.axhline(y=1, color='k',lw=1)
plt.legend()
plt.show()

In [ ]:
# Plot simulation results
for tempIN, colorIN, noteIN in zip(temperatures, color_list_rgba, notes):    
    key = f"PE_{tempIN}K_{noteIN}"
    
    # Plot E-field magnitude at target location
    plt.semilogy(results_data[key]["fieldAtTarget"]["iter"],
             np.array(results_data[key]["gradRefinements"])/1e6,
             '.-',
             label=f"{tempIN} K: {noteIN}",
             lw=0.5, 
             color=colorIN)
plt.xlabel("Iteration #")
plt.ylabel("# of Gradient Refinements (millions)")
plt.title("PE Case")
plt.axhline(y=1, color='k',lw=1)
plt.legend()
plt.show()

In [ ]:
## test one file ##

# configIN = "onlyphotoemission"
# directory_path = "../build-disspate-charge/"

# filenames = sorted(glob.glob(f"{directory_path}/fieldmaps/*{configIN}*.txt"))
# fields_PE = read_data_format_efficient(filenames,scaling=True) 

# # Target point
# location = np.array([-0.1, 0, 0.1-0.015+0.037]) 
# # return the electric field at that location
# Efield_PE_location = compute_nearest_field_vector(fields_PE, target=location, start=1)

In [ ]:
## process different sets of iterations (not in parallel) ##

# configIN = "onlyphotoemission"
# folder_path = ["../build-temp425K-dynamicThreshold",  "../build-temp600K-dynamicThreshold", "../build-425K-initial6-0.9Max-0.05um-final10"]
# temperatures = [425,600,425]
# notes = ["initial6max0.2final9", "initial6max0.2final9", "initial6max0.6final10"]

# # Target point
# location = np.array([-0.1, 0, 0.1-0.015+0.037]) 

# # MASTER DICTIONARY to store all results securely
# results_data: Dict[str, Dict[str, Any]] = {}

# # --- Processing Loop ---

# # Use enumerate for index 'j' and zip the parameter lists together
# for j, (tempIN, folderIN, noteIN) in enumerate(zip(temperatures, folder_path, notes)):

#     # Create a unique key for storing results
#     key_name = f"PE_{tempIN}K_{noteIN}"
    
#     print(f"--- Processing {j}: {key_name} from {folderIN} ---")

#     # 1. Read Field Data
#     filenames = sorted(glob.glob(f"{folderIN}/fieldmaps/*{configIN}*.txt"))
#     fields_PE = read_data_format_efficient(filenames, scaling=True) 
#     first_key = list(fields_PE.keys())[0]
    
#     # 2. Initialize entry in the results dictionary
#     results_data[key_name] = {}
    
#     # 3. Compute and Store Field at Target Location
#     results_data[key_name]["fieldAtTarget"] = compute_nearest_field_vector(fields_PE, target=location, start=first_key)
    
#     # 4. Compute and Store the list of leaf lengths (number of points per iteration)
#     results_data[key_name]["lengthLeaves"] = np.array([len(fields_PE[keyIN]["pos"]) for keyIN in fields_PE.keys()])
    
#     # If the fields_PE dictionary is very large, deleting it immediately frees memory
#     #del fields_PE 

### check the minimum distance between point in field map 

In [ ]:
## test one file ##

directory_path = "../build-smallerworld-initial8max0.8final13/"
filenames = sorted(glob.glob(f"{directory_path}/fieldmaps/*{configIN}*.txt"))
fields_PE = read_data_format_efficient(filenames,scaling=True) 

In [ ]:
# Assuming 'fields_PE' is your list/dictionary structure and 
# fields_PE[1]['pos'] is a NumPy array of shape (N, 3), where N is the number of points.
# Example data (replace this with your actual data):
data_points = fields_PE[1]['pos'] 
# data_points = np.array([
#     [1.0, 1.0, 1.0],
#     [1.001, 1.0, 1.0],  # Very close point
#     [2.0, 2.0, 2.0],
#     [5.0, 5.0, 5.0]
# ], dtype=np.float32)

# 1. Build the KD-Tree
# This organizes the points in a spatial structure for efficient nearest neighbor search.
tree = cKDTree(data_points)

# 2. Query for the 2 nearest neighbors of every point
# The 'k=2' parameter tells the query to find the distance to the 2 nearest neighbors:
# - The 1st neighbor (k=1) is always the point itself (distance = 0.0).
# - The 2nd neighbor (k=2) is the closest *other* point.
distances, indices = tree.query(data_points, k=2)

# 3. Extract the minimum non-zero distance
# The minimum distance between any unique pair of points is the minimum value 
# in the array of distances to the second nearest neighbor (distances[:, 1]).
min_distance = np.min(distances[:, 1])*1000

print(f"The total number of points (voxels) is: {len(data_points)}")
print(f"The minimum distance between any two unique voxels is: {min_distance} um")

# takes around ~10 seconds to run


### calculate temperature change over all iterations:

In [ ]:
# --- Worker Function for Parallel Execution ---
def process_root_file(fileIN: str, directory_path: str, target_volume = "SiO2") -> Tuple[int, float]:
    """Reads a single ROOT file, performs event analysis, and returns the index and calculated total energy."""

    number_str = fileIN.split("/")[-1].split("_")[1]
    iterationNUM = int(''.join(filter(str.isdigit, number_str)))

    try:
        # Read data for the current iteration
        df = read_rootfile(fileIN.split("/")[-1], directory_path=directory_path)

    except Exception:
        # Catch errors like missing keys or file corruption during read_rootfile
        print(f"-> ERROR: Skipping {fileIN.split('/')[-1]} due to failed file read")
        return iterationNUM, 0.0 # Return 0 energy and the index to maintain order

    # 1. Get all incident gamma events (Particle_Type="gamma", Parent_ID=0.0)
    incident_gamma = df[(df["Particle_Type"] == "gamma") & (df["Parent_ID"] == 0.0)].drop_duplicates(subset="Event_Number", keep="first")

    # 2. Get all unique event numbers that resulted in an electron creation
    last_e_event = df[(df["Particle_Type"] == "e-") & (df["Parent_ID"] > 0.0)].drop_duplicates(subset="Event_Number", keep="last")
    event_numbers_with_e_creation = last_e_event["Event_Number"].unique()

    # 3. All unique incident gamma event numbers
    incident_gamma_event_numbers = incident_gamma["Event_Number"].unique()

    # 4. Events where NO electron was created (incident gamma events - events with e- creation)
    events_without_photoelectric_e = np.setdiff1d(incident_gamma_event_numbers, event_numbers_with_e_creation)

    # 5. Filter the main DataFrame to contain only data from the non-interacting events
    events_without_photoelectric_e_df = df[df["Event_Number"].isin(events_without_photoelectric_e)]

    # 6. Calculate total energy deposited by gammas that *did not* result in a photoelectric electron
    totalEnergy = np.sum(events_without_photoelectric_e_df[
        (events_without_photoelectric_e_df["Particle_Type"] == "gamma") & 
        (events_without_photoelectric_e_df["Volume_Name_Post"] == target_volume)
    ]["Kinetic_Energy_Diff_eV"])

    print(f"-> PROCESSED #{iterationNUM}: {fileIN.split('/')[-1]}")

    return iterationNUM, totalEnergy

# --- Configuration ---
configIN = "onlyphotoemission"
directory_path =  "../build-temp425K-dynamicThreshold/root/" 
filelist = sorted(glob.glob(f"{directory_path}/*iteration*{configIN}*.root"))

# --- Main Parallel Execution ---

# List to hold the (index, totalEnergy) tuples from parallel processes
NUM_FILES = len(filelist)
photoEnergyDepositionsforIterations = np.empty(NUM_FILES, dtype=np.float64)

print(f"--- Starting Parallel Processing of {NUM_FILES} files ---")

with concurrent.futures.ProcessPoolExecutor(max_workers=NUM_FILES) as executor:
    
    # Submit tasks, passing the index to ensure results are ordered correctly later
    futures = [executor.submit(process_root_file, fileIN, directory_path) for fileIN in filelist]
    
    # Collect results as they complete
    for future in concurrent.futures.as_completed(futures):
        index, totalEnergy = future.result()
        photoEnergyDepositionsforIterations[index]= totalEnergy

# Final assignment to the NumPy array
#photoEnergyDepositionsforIterations = np.array([r[1] for r in all_results], dtype=np.float64)

print("\nProcessing complete.")


In [ ]:
# define constants
initialT = 425
heat_capacity = 670+1e3*((initialT-250)/530.6)-1e3*((initialT-250)/498.7)**2 # for lunar regolith
density = 2.2/1000 #kg/cm3
radius = 5 # assuming that all of the energy is deposited in a 10 um area!!
volume = 4/2*np.pi*(radius*1e-4)**3 # cm3
mass = volume*density # mass of material
# this radius and volume is not true, just calculated as an extreme to see if we need to dynamically adjust the temperature!!

print("--- Over {NUM_FILES} Iterations ---")
print(f"Mean Temperature Increase : {np.mean(photoEnergyDepositionsforIterations*1.60218e-19/heat_capacity/mass*100)} K")
print(f"Total Temperature Increase: {np.sum(photoEnergyDepositionsforIterations*1.60218e-19/heat_capacity/mass*100)} K")

### create plots of the face ilumination

In [ ]:
configIN = "onlysolarwind"
#directory_path =  "../build-temp425K-dynamicThreshold/root/" #"../build-adaptive-barns-fixed/root/"
#directory_path = "/storage/scratch1/5/avira7/Grain-Charging-Simulation-Data/stacked-sphere/output110525/smallerworld-initial8max0.8final12/"
directory_path = "../build-initial8max0.7final11-wang/root/"

filelist = sorted(glob.glob(f"{directory_path}/*iteration*{configIN}*_num50000.root"))

all_protons_inside = []
all_electrons_inside = []

for fileIN in filelist:
    print(fileIN.split("/")[-1])
    number_str = fileIN.split("/")[-1].split("_")[1]
    iterationNUM = int(''.join(filter(str.isdigit, number_str)))

    if iterationNUM > 20:
        break

    # read data from different iterations
    electrons_df, protons_df = calculate_stats(read_rootfile(fileIN.split("/")[-1], directory_path=directory_path), config=configIN)

    all_protons_inside.append(protons_df)
    all_electrons_inside.append(electrons_df)
    print(78*"-")

In [ ]:
# Concatenate all iterations into single DataFrames
all_protons_inside_df = pd.concat(all_protons_inside, ignore_index=True)
all_electrons_inside_df = pd.concat(all_electrons_inside, ignore_index=True)

In [ ]:
save_directory = "/storage/coda1/p-zjiang33/0/shared/avira7/root_files/stacked-sphere/processed-files" 
all_protons_inside_df.to_pickle(f'{save_directory}/SW_protons_locations_until20.pkl') # only got through 43 iterations
all_electrons_inside_df.to_pickle(f'{save_directory}/SW_electrons_locations_until20.pkl')

In [ ]:
surf, ilm_values,_ = plot_face_illumination(gamma_holes_iteration0, stacked_spheres, vmin=0, vmax=5)
# Make sure each triangle has its own unique vertices
surface_edited = surf.copy()
surface_edited.unmerge_vertices()
surface_edited.visual.vertex_colors = None
surface_edited.show()

In [ ]:
configIN = "onlyphotoemission"
directory_path = "/storage/scratch1/5/avira7/Grain-Charging-Simulation-Data/stacked-sphere/output110525/smallerworld-initial8max0.8final12/root/"
filelist = sorted(glob.glob(f"{directory_path}/*{configIN}*_num100000.root"))

all_gamma_holes = []
all_electrons_inside = []

for fileIN in filelist:
    print(fileIN.split("/")[-1])
    number_str = fileIN.split("/")[-1].split("_")[1]
    iterationNUM = int(''.join(filter(str.isdigit, number_str)))

    if iterationNUM < 80:
        continue

    # read data from different iterations
    gamma_holes_df, electron_inside_df, _ = calculate_stats(read_rootfile(fileIN.split("/")[-1], directory_path=directory_path),
                                                                                                             config=configIN)
    all_gamma_holes.append(gamma_holes_df)
    all_electrons_inside.append(electron_inside_df)
    print(78*"-")


# Concatenate all iterations into single DataFrames
all_gamma_holes_df = pd.concat(all_gamma_holes, ignore_index=True)
all_electrons_inside_df = pd.concat(all_electrons_inside, ignore_index=True)

In [ ]:
# Concatenate all iterations into single DataFrames
all_gamma_holes_df = pd.concat(all_gamma_holes, ignore_index=True)
all_electrons_inside_df = pd.concat(all_electrons_inside, ignore_index=True)

In [ ]:
save_directory = "/storage/coda1/p-zjiang33/0/shared/avira7/root_files/stacked-sphere/processed-files" 
all_gamma_holes_df.to_pickle(f'{save_directory}/PE_gammaholes_locations.pkl') # only got through 43 iterations
all_electrons_inside_df.to_pickle(f'{save_directory}/PE_electronsinside_locations.pkl')

In [ ]:
def load_pickle(filepath):
    """
    Loads data from a pickle file.
    """
    if not os.path.exists(filepath):
        print(f"Error: File not found at {filepath}")
        return None
        
    try:
        with open(filepath, 'rb') as f:
            data = pickle.load(f)
        print(f"Successfully loaded data from {filepath}")
        return data
    except Exception as e:
        print(f"Error loading pickle file {filepath}: {e}")
        return None

In [ ]:
import pickle
save_directory = "/storage/coda1/p-zjiang33/0/shared/avira7/root_files/stacked-sphere/processed-files" 
all_gamma_holes_df=load_pickle(f'{save_directory}/PE_gammaholes_locations.pkl') # only got through 43 iterations
all_electrons_inside_df=load_pickle(f'{save_directory}/PE_electronsinside_locations.pkl')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import trimesh
 
# --- Define physical constants in SI units ---
epsilon_0 = 8.854187817e-12  # F/m (Permittivity of free space)
q_electron = -1.602176634e-19 # C
q_proton = +1.602176634e-19   # C
 
def plot_surface_Txx_from_charge_density(electrons, protons, convex_combined, vmin=-0.01, vmax=0.01):
    """
    Calculates and plots the xx-component of the Maxwell Stress Tensor
    T_xx = (sigma^2 / epsilon_0) * (n_x^2 - 0.5)
    by first binning particles to faces to find sigma.
 
    Args:
        electrons (pd.DataFrame): DataFrame with 'Pre_Step_Position_mm'.
        protons (pd.DataFrame): DataFrame with 'Pre_Step_Position_mm'.
        convex_combined (trimesh.Trimesh): The mesh object (assumed to be in mm).
        vmin (float): Minimum stress (in Pascals) for the color map.
        vmax (float): Maximum stress (in Pascals) for the color map.
 
    Returns:
        (trimesh.Trimesh, np.array, np.array):
        The mesh object with face_colors set,
        the 1D array of T_xx per face (in Pascals),
        the RGBA color array.
    """
 
    # --- 1. Initialize Charge and Area Arrays ---
    num_faces = len(convex_combined.faces)
    # This array will hold the total charge (Q) on each face in Coulombs
    face_charges = np.zeros(num_faces)
    # Get face areas (in mm^2) and convert to m^2
    # (1 mm)^2 = (1e-3 m)^2 = 1e-6 m^2
    face_areas_m2 = convex_combined.area_faces * 1e-6
    # Add a small epsilon to prevent division by zero for any zero-area faces
    face_areas_m2 += 1e-20
 
    # --- 2. Get Particle Positions (in mm) ---
    e_pos = np.array(electrons["Post_Step_Position_mm"].tolist())
    p_pos = np.array(protons["Post_Step_Position_mm"].tolist())
 
    # --- 3. Bin Electrons and Protons to Faces to get Q_face ---
    # Bin electrons
    if len(e_pos) > 0:
        _, _, face_id_e = convex_combined.nearest.on_surface(e_pos)
        unique_faces_e, counts_e = np.unique(face_id_e, return_counts=True)
        face_charges[unique_faces_e] += (counts_e * q_electron)
 
    # Bin protons
    if len(p_pos) > 0:
        _, _, face_id_p = convex_combined.nearest.on_surface(p_pos)
        unique_faces_p, counts_p = np.unique(face_id_p, return_counts=True)
        face_charges[unique_faces_p] += (counts_p * q_proton)
 
    # --- 4. Calculate Surface Charge Density (sigma) ---
    # sigma = Q_face / A_face (in C/m^2)
    sigma_per_face = face_charges / face_areas_m2
    # Calculate the sigma^2 / epsilon_0 term (in Pascals, N/m^2)
    sigma_sq_over_eps = (sigma_per_face**2) / epsilon_0
 
    # --- 5. Calculate T_xx Component ---
    # Get the (N_faces, 3) array of unit normal vectors
    face_normals = convex_combined.face_normals
    # Get the x-component of each normal vector, n_x
    # (n_dot_x) is just normals[:, 0]
    n_x = face_normals[:, 0]
    # Calculate the (n_x^2 - 0.5) term
    geometry_term = (n_x**2) - 0.5
    # Calculate the final T_xx for each face
    face_Txx_values = sigma_sq_over_eps * geometry_term
 
    # --- 6. Apply Colormap ---
    colors_rgba = np.zeros((num_faces, 4), dtype=np.uint8)
    # T_xx can be positive or negative, so 'seismic' is the correct colormap
    cmap = plt.cm.seismic
    norm_func = Normalize(vmin=vmin, vmax=vmax)
    colors_rgb = cmap(norm_func(face_Txx_values))[:, :3]
 
    # Assign RGB to all faces
    colors_rgba[:, :3] = (colors_rgb * 255).astype(np.uint8)
    # Set alpha to opaque so faces are visible
    colors_rgba[:, 3] = 255
 
    # Apply colors to mesh
    convex_combined.visual.face_colors = colors_rgba
 
    return convex_combined, face_Txx_values, colors_rgba

In [ ]:
print("up through iteration 3")
# input order: gammas, photoelectrons, protons, electrons, convex_combined,
surface,facecolors,_ = plot_surface_Txx_from_charge_density(all_electrons_inside_df, all_gamma_holes_df, stacked_spheres, vmin=-0.01,vmax=0.01)
print(min(facecolors),max(facecolors))
plt.plot(facecolors[facecolors!=0])
plt.show()

# Make sure each triangle has its own unique vertices
surface_edited = surface.copy()
surface_edited.unmerge_vertices()
surface_edited.visual.vertex_colors = None
surface_edited.show()

In [ ]:
def plot_face_illumination(incident_particles, convex_combined, vmin=0, vmax=1):
    """
    Colors the mesh faces based on how many particles are nearest to each face.
    """
    # Step 1: Get particle positions as Nx3 array
    points = np.array(incident_particles["Post_Step_Position_mm"].tolist())

    # Step 2: Get nearest face indices
    _, _, face_ids = convex_combined.nearest.on_surface(points)

    # Step 3: Accumulate per-face "illumination"
    face_illum = np.zeros(len(convex_combined.faces))
    np.add.at(face_illum, face_ids, 1)

    # Step 4: Normalize face illumination (optional)
    norm_f = face_illum.astype(float)

    # Step 5: Apply colormap
    cmap = plt.cm.OrRd
    norm_func = Normalize(vmin=vmin, vmax=vmax)
    rgb_f = cmap(norm_func(norm_f))[:, :3]

    # Step 6: Build RGBA face colors
    colors_f = np.zeros((len(face_illum), 4), dtype=np.uint8)
    colors_f[:, :3] = (rgb_f * 255).astype(np.uint8)
    colors_f[:, 3] = 255  # Fully opaque

    # Step 7: Apply colors to faces
    convex_combined.visual.face_colors = colors_f

    return convex_combined, face_illum, colors_f

In [ ]:
surf, ilm_values,_ = plot_face_illumination(all_electrons_inside_df, stacked_spheres, vmin=0, vmax=400)
print(max(ilm_values))

# Make sure each triangle has its own unique vertices
surface_edited = surf.copy()
surface_edited.unmerge_vertices()
surface_edited.visual.vertex_colors = None
surface_edited.show()

In [ ]:
def exp_saturation(t, a, b, tau):
    return a - b * np.exp(-t / tau)


# Initial parameter guess: a, b, tau
initial_guess = [4.0, 2.0, 1.0]
popt, pcov = curve_fit(exp_saturation, 10**zimmerman_PEdata["x"], zimmerman_PEdata[" y"], p0=initial_guess)

# Plot
plt.plot(10**zimmerman_SWdata["x"], zimmerman_SWdata[" y"],'--',label="SW",color="r",lw=4)
plt.plot(10**zimmerman_PEdata["x"], zimmerman_PEdata[" y"],'g:',label="PE")
plt.plot(10**zimmerman_PEandSWdata["x"], zimmerman_PEandSWdata[" y"],'b:',label="PE+SW")
plt.plot((efield_PE["iter"] - 101)*convert_iteration_PEtime, abs(mag_values_PE), 'k.-',label="Geant4: PE (different incident)",lw=0.5)
#plt.semilogx((efield_PE_noholes["iter"] - 101)*convert_iteration_PEtime2, abs(mag_values_PE_noholes), 'b.-',label="Geant4: PE (no holes)",lw=0.5)
#plt.plot((efield_PE_normal["iter"] - 101)*convert_iteration_PEtime, abs(mag_values_PE_normal), 'g.-',label="Geant4: PE",lw=0.5)
#plt.plot((efield_SW["iter"] - 1)*convert_iteration_SWtime, mag_values_SW, 'k.-',label="Geant4: SW",lw=0.5)

# xdata_zimmerman = np.linspace(1e-2,10,100)
# y_fit_zimmerman = exp_saturation(xdata_zimmerman, *popt)
# #plt.plot(xdata_zimmerman, y_fit_zimmerman, 'r-',lw=0.2)

# # Initial parameter guess: a, b, tau
# initial_guess = [4.0, 2.0, 1.0]
# popt, pcov = curve_fit(exp_saturation,(efield_PE["iter"] - 101)*convert_iteration_PEtime, abs(mag_values_PE), p0=popt)

# xdata = np.linspace(1e-1,5,100)
# y_fit = exp_saturation(xdata, *popt)
# plt.plot(xdata, y_fit, 'g-',lw=0.2, label="prediction")
# #plt.plot(xdata_zimmerman, y_fit_zimmerman-3.8e4, 'r-',lw=0.2)
# plt.axvline(x=4)

plt.xlabel("Time [s]")
plt.ylabel(r"|E| (V/m)")
#plt.ylabel(r"E$_x$ (V/m)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
plt.hist(ilm_values, bins=np.logspace(-1,3,100))
plt.xscale("log")
plt.title("# of Photons hitting each Voxel")
plt.show()

print(f"Distribution Stats: mean {np.mean(ilm_values[ilm_values!=0])}, median {np.median(ilm_values[ilm_values!=0])}, max {np.max(ilm_values)}")

# our area is a factor of 4 smaller than their area 
print(f"for one iteration, median # of particles in each equivalently sized voxel is {np.median(ilm_values[ilm_values!=0])/4}")
voxel_area = 0.0004/(1000)**2 # rough area approximated from python (0.4 micron2)
zimmerman_charge = 0.5*(1e-6) # C/m2
zimmerman_electronnum = (zimmerman_charge/1.60217663e-19)*voxel_area
print(f"will take {zimmerman_electronnum/(np.max(ilm_values[ilm_values!=0])/4)} iterations at this rate to get to the photoemission flux ranges shown at 3 seconds")

In [ ]:
print("up through iteration 33")
surface,pot = plot_surface_potential_fornegativepositive_charge(all_electrons_inside_df, all_gamma_holes_df, stacked_spheres, vmin=-0.2,vmax=0.2)
print(min(pot),max(pot))
# Make sure each triangle has its own unique vertices
surface_edited = surface.copy()
surface_edited.unmerge_vertices()
surface_edited.visual.vertex_colors = None
surface_edited.show()

In [ ]:
bbox_min = np.array([-0.2, -0.2, 0])
bbox_max = np.array([ 0.2,  0.2,  100])

# Filter vertices
in_box = np.all((stacked_spheres.vertices >= bbox_min) & 
                (stacked_spheres.vertices <= bbox_max), axis=1)

# Get face indices where all 3 vertices are in the box
face_mask = np.all(in_box[stacked_spheres.faces], axis=1)

# Extract submesh
cropped = stacked_spheres.submesh([face_mask], only_watertight=False, append=True)
cropped.show()

In [ ]:
# Define colormap and normalization
cmap = plt.cm.seismic
norm = Normalize(vmin=-0.2, vmax=0.2)

# Create a figure and a single axis for the colorbar
fig, ax = plt.subplots(figsize=(4, 0.5))
fig.subplots_adjust(bottom=0.5)

# Create the colorbar
cb = ColorbarBase(ax, cmap=cmap, norm=norm, orientation='horizontal')
cb.set_label('Surface Potential (mV)')  # Optional label

plt.show()


## Case 3: all particles (incident e-, protons, gammas)

In [ ]:
directory_path = "../build-sphere-charging/root/"
configIN = "allparticles"
filelist = sorted(glob.glob(f"{directory_path}/*stackediteration*{configIN}*num5000.root"))

all_gamma_holes,all_photoemission_electrons,all_protons_inside,all_electrons_inside = [],[],[],[]

for fileIN in filelist:
    print(fileIN.split("/")[-1])
    number_str = fileIN.split("/")[-1].split("_")[1]
    iterationNUM = int(''.join(filter(str.isdigit, number_str)))

    # read data from different iterations
    vars()["gamma_holes_"+str(number_str)], vars()["photoemission_electrons_inside_"+str(number_str)], \
        vars()["protons_inside_"+str(number_str)], vars()["electrons_inside_"+str(number_str)] = calculate_stats(read_rootfile(fileIN.split("/")[-1], directory_path=directory_path), \
                                                                                                             config=configIN)
    all_gamma_holes.append(vars()["gamma_holes_"+str(number_str)])
    all_photoemission_electrons.append(vars()["photoemission_electrons_inside_"+str(number_str)])
    all_protons_inside.append(vars()["protons_inside_"+str(number_str)])
    all_electrons_inside.append(vars()["electrons_inside_"+str(number_str)])
    print(78*"-")

# Concatenate all iterations into single DataFrames
all_gamma_holes_df = pd.concat(all_gamma_holes, ignore_index=True)
all_photoemission_electrons_df = pd.concat(all_photoemission_electrons, ignore_index=True)
all_protons_inside_df = pd.concat(all_protons_inside, ignore_index=True)
all_electrons_inside_df = pd.concat(all_electrons_inside, ignore_index=True)

In [ ]:
# Concatenate all iterations into single DataFrames
all_gamma_holes_df = pd.concat(all_gamma_holes, ignore_index=True)
all_photoemission_electrons_df = pd.concat(all_photoemission_electrons, ignore_index=True)
all_protons_inside_df = pd.concat(all_protons_inside, ignore_index=True)
all_electrons_inside_df = pd.concat(all_electrons_inside, ignore_index=True)

In [ ]:
print("up through iteration 28")
# input order: gammas, photoelectrons, protons, electrons, convex_combined,
surface,facecolors = plot_electric_pressure_from_charge_density(all_gamma_holes_df, all_photoemission_electrons_df, all_protons_inside_df, all_electrons_inside_df, 
                                                      stacked_spheres, vmin=-0.2,vmax=0.2)
print(min(facecolors),max(facecolors))
plt.plot(facecolors[facecolors!=0])
plt.show()

# Make sure each triangle has its own unique vertices
surface_edited = surface.copy()
surface_edited.unmerge_vertices()
surface_edited.visual.vertex_colors = None
surface_edited.show()

In [ ]:
bbox_min = np.array([-0.2, -0.3, 0])
bbox_max = np.array([ 0.2,  0.1,  100])

# Filter vertices
in_box = np.all((surface_edited.vertices >= bbox_min) & 
                (surface_edited.vertices <= bbox_max), axis=1)

# Get face indices where all 3 vertices are in the box
face_mask = np.all(in_box[surface_edited.faces], axis=1)

# Extract submesh
cropped = surface_edited.submesh([face_mask], only_watertight=False, append=True)
cropped.show()

In [ ]:
# Define colormap and normalization
cmap = plt.cm.seismic
norm = Normalize(vmin=-0.2, vmax=0.2)

# Create a figure and a single axis for the colorbar
fig, ax = plt.subplots(figsize=(4, 0.5))
fig.subplots_adjust(bottom=0.5)

# Create the colorbar
cb = ColorbarBase(ax, cmap=cmap, norm=norm, orientation='horizontal')
cb.set_label('Surface Potential (mV)')  # Optional label

plt.show()


In [ ]:
print("up through iteration 19: only SW ions")
surface,facecolors = plot_surface_potential_fornegativepositive_charge(all_electrons_inside_df, all_protons_inside_df, stacked_spheres, vmin=-0.2,vmax=0.2)

print(min(facecolors),max(facecolors))
plt.plot(facecolors[facecolors!=0])
plt.show()

# Make sure each triangle has its own unique vertices
surface_edited = surface.copy()
surface_edited.unmerge_vertices()
surface_edited.visual.vertex_colors = None
surface_edited.show()

In [ ]:
print("up through iteration 22: only photons")
surface,facecolors = plot_surface_potential_fornegativepositive_charge(all_photoemission_electrons_df, all_gamma_holes_df, stacked_spheres, vmin=-0.5,vmax=0.5)

print(min(facecolors),max(facecolors))
plt.plot(facecolors[facecolors!=0])
plt.show()

# Make sure each triangle has its own unique vertices
surface_edited = surface.copy()
surface_edited.unmerge_vertices()
surface_edited.visual.vertex_colors = None
surface_edited.show()

In [ ]:
bbox_min = np.array([-0.2, -0.3, 0])
bbox_max = np.array([ 0.2,  0.1,  100])

# Filter vertices
in_box = np.all((surface_edited.vertices >= bbox_min) & 
                (surface_edited.vertices <= bbox_max), axis=1)

# Get face indices where all 3 vertices are in the box
face_mask = np.all(in_box[surface_edited.faces], axis=1)

# Extract submesh
cropped = surface_edited.submesh([face_mask], only_watertight=False, append=True)
cropped.show()
